#**Capstone Project - Colon Cancer**

###DataSet

Get Kaggle Dataset into GoogleColab

##**Code - Colon Cancer**

###**AlexNet**

Import all libraries required

In [ ]:
!pip install xgboost
import numpy as np
import pandas as pd
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score, confusion_matrix
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout
from keras.models import Sequential
import tensorflow as tf
import matplotlib.pyplot as plt
import cv2
from tensorflow.keras.layers import BatchNormalization
from keras.models import Model, Sequential
from keras.applications.xception import Xception
from keras.applications import *
import matplotlib.pyplot as plt
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline
from PIL import Image
import random
import os
import cv2
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from keras.callbacks import EarlyStopping
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm
from sklearn.decomposition import PCA

In [ ]:
data_dir = "colon_image_sets"
SIZE_X = SIZE_Y = 227

datagen = tf.keras.preprocessing.image.ImageDataGenerator(validation_split = 0.3)

train_it = datagen.flow_from_directory(data_dir,
                                       class_mode = "categorical",
                                       target_size = (SIZE_X,SIZE_Y),
                                       color_mode="rgb",
                                       batch_size = 12,
                                       shuffle = False,
                                       subset='training',
                                       seed = 42)

validate_it = datagen.flow_from_directory(data_dir,
                                       class_mode = "categorical",
                                       target_size = (SIZE_X, SIZE_Y),
                                       color_mode="rgb",
                                       batch_size = 12,
                                       shuffle = False,
                                       subset='validation',
                                       seed = 42)

Found 7000 images belonging to 2 classes.
Found 3000 images belonging to 2 classes.


In [ ]:
def get_features(base_model, train, validate):
    X_train = base_model.predict(train)
    y_train = train.classes

    X_val = base_model.predict(validate)
    y_val = validate.classes

    X_val, X_test, y_val, y_test = train_test_split(X_val, y_val, test_size = 0.5, shuffle = True)
    print('Shape of X_train----->', str(X_train.shape))
    print('Shape of X_val----->', str(X_val.shape))
    print('Shape of X_test----->', str(X_test.shape))
    return (X_train, X_val, X_test, y_train, y_val, y_test)

In [ ]:
def get_models():
    ANN = Sequential()
    ANN.add(Dense(128, input_dim = X_train.shape[1], activation = 'relu'))
    ANN.add(BatchNormalization())
    ANN.add(Dropout(0.2))
    ANN.add(Dense(64, activation='relu'))
    ANN.add(Dense(32, activation='relu'))
    ANN.add(Dense(16, activation='relu'))
    ANN.add(Dense(8, activation='relu'))
    ANN.add(Dense(len(train_it.class_indices), activation='softmax'))
    ANN.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    LR = LogisticRegression()

    KNN = KNeighborsClassifier()

    SVM = SVC(kernel = 'linear')

    RF = RandomForestClassifier(n_estimators = 50)

    XGB = XGBClassifier(n_estimators = 50, use_label_encoder=False)

    return (ANN, LR, KNN, SVM, RF, XGB)

In [ ]:
def reshape_data(X_train, X_val, X_test):
    X_train = X_train.reshape(X_train.shape[0], -1)
    X_val = X_val.reshape(X_val.shape[0], -1)
    X_test = X_test.reshape(X_test.shape[0], -1)

    print("Shape after reshaping------->")
    print("X train------->", str(X_train.shape))
    print("X val-------->", str(X_val.shape))
    print("X test-------->", str(X_test.shape))

    return (X_train, X_val, X_test)

In [ ]:
def fit_ANN(model, X_train, y_train, X_val, y_test):
    es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)
    history = model.fit(X_train, y_train, validation_data=(X_val, y_test), epochs=10, verbose=1, callbacks=[es])
    return model

def fit_model(model, X_train, y_train):
    model.fit(X_train, y_train)
    return model

In [ ]:
from keras.models import Model
from keras.layers import Input, Flatten, Dense, Conv2D, MaxPooling2D

def create_alexnet(input_shape):
    input_layer = Input(shape=input_shape)

    # First Convolutional Layer
    x = Conv2D(filters=96, kernel_size=(11,11), strides=(4,4), activation='relu')(input_layer)
    x = MaxPooling2D(pool_size=(3,3), strides=(2,2))(x)

    # Second Convolutional Layer
    x = Conv2D(filters=256, kernel_size=(5,5), strides=(1,1), padding='same', activation='relu')(x)
    x = MaxPooling2D(pool_size=(3,3), strides=(2,2))(x)

    # Three Convolutional Layers
    x = Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu')(x)
    x = Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu')(x)
    x = Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu')(x)
    x = MaxPooling2D(pool_size=(3,3), strides=(2,2))(x)

    # Flatten Layer
    x = Flatten()(x)

    # Fully Connected Layers
    x = Dense(units=4096, activation='relu')(x)
    x = Dense(units=4096, activation='relu')(x)
    output_layer = Dense(units=1000, activation='softmax')(x)  # Assuming 1000 classes for ImageNet

    model = Model(inputs=input_layer, outputs=output_layer)
    return model

# Assuming SIZE_X and SIZE_Y are defined elsewhere
model = create_alexnet(input_shape=(SIZE_X, SIZE_Y, 3))
model.summary()


Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 227, 227, 3)]     0         
                                                                 
 conv2d_15 (Conv2D)          (None, 55, 55, 96)        34944     
                                                                 
 max_pooling2d_9 (MaxPooling  (None, 27, 27, 96)       0         
 2D)                                                             
                                                                 
 conv2d_16 (Conv2D)          (None, 27, 27, 256)       614656    
                                                                 
 max_pooling2d_10 (MaxPoolin  (None, 13, 13, 256)      0         
 g2D)                                                            
                                                                 
 conv2d_17 (Conv2D)          (None, 13, 13, 384)       8851

In [ ]:
X_train, X_val, X_test, y_train, y_val, y_test = get_features(model, train_it, validate_it)

Shape of X_train-----> (7000, 1000)
Shape of X_val-----> (1500, 1000)
Shape of X_test-----> (1500, 1000)


In [ ]:
X_train, X_val, X_test = reshape_data(X_train, X_val, X_test)
ANN, LR, KNN, SVM, RF, XGB = get_models()

from sklearn.model_selection import KFold

# Define number of splits for K-Fold cross-validation
k_folds = 5
kf = KFold(n_splits=k_folds, shuffle=True)

# Initialize lists to store performance metrics for each fold
accuracy_scores_ANN = []
f1_scores_ANN = []
accuracy_scores_LR = []
f1_scores_LR = []
accuracy_scores_KNN = []
f1_scores_KNN = []
accuracy_scores_SVM = []
f1_scores_SVM = []
accuracy_scores_RF = []
f1_scores_RF = []
accuracy_scores_XGB = []
f1_scores_XGB = []
# Perform K-Fold cross-validation
for train_index, val_index in kf.split(X_train):
    X_train_fold, X_val_fold = X_train[train_index], X_train[val_index]
    y_train_fold, y_val_fold = y_train[train_index], y_train[val_index]

    # Train ANN model
    ANN_fold = fit_ANN(ANN, X_train_fold, y_train_fold, X_val_fold, y_val_fold)
    # Evaluate ANN model
    accuracy_ANN = accuracy_score(y_test, np.argmax(ANN_fold.predict(X_test), axis=1))
    f1_ANN = f1_score(y_test, np.argmax(ANN_fold.predict(X_test), axis=1))
    accuracy_scores_ANN.append(accuracy_ANN)
    f1_scores_ANN.append(f1_ANN)

    # Train LR model
    LR_fold = fit_model(LR, X_train_fold, y_train_fold)
    # Evaluate LR model
    accuracy_LR = accuracy_score(y_test, LR_fold.predict(X_test))
    accuracy_scores_LR.append(accuracy_LR)
    f1_LR=f1_score(y_test, LR_fold.predict(X_test))
    f1_scores_LR.append(f1_LR)

    # Train KNN model
    KNN_fold = fit_model(KNN, X_train_fold, y_train_fold)
    # Evaluate KNN model
    accuracy_KNN = accuracy_score(y_test, KNN_fold.predict(X_test))
    accuracy_scores_KNN.append(accuracy_KNN)
    f1_KNN=f1_score(y_test, KNN_fold.predict(X_test))
    f1_scores_KNN.append(f1_KNN)

    # Train SVM model
    SVM_fold = fit_model(SVM, X_train_fold, y_train_fold)
    # Evaluate SVM model
    accuracy_SVM = accuracy_score(y_test, SVM_fold.predict(X_test))
    accuracy_scores_SVM.append(accuracy_SVM)
    f1_SVM=f1_score(y_test, SVM_fold.predict(X_test))
    f1_scores_SVM.append(f1_SVM)

    # Train RF model
    RF_fold = fit_model(RF, X_train_fold, y_train_fold)
    # Evaluate RF model
    accuracy_RF = accuracy_score(y_test, RF_fold.predict(X_test))
    accuracy_scores_RF.append(accuracy_RF)
    f1_RF=f1_score(y_test, RF_fold.predict(X_test))
    f1_scores_RF.append(f1_RF)

    # Train XGB model
    XGB_fold = fit_model(XGB, X_train_fold, y_train_fold)
    # Evaluate XGB model
    accuracy_XGB = accuracy_score(y_test, XGB_fold.predict(X_test))
    accuracy_scores_XGB.append(accuracy_XGB)
    f1_XGB=f1_score(y_test, XGB_fold.predict(X_test))
    f1_scores_XGB.append(f1_XGB)

Shape after reshaping------->
X train-------> (7000, 1000)
X val--------> (1500, 1000)
X test--------> (1500, 1000)
Epoch 1/10
175/175 [==============================] - 1s 4ms/step - loss: 0.6769 - accuracy: 0.5716 - val_loss: 0.6859 - val_accuracy: 0.6200
Epoch 2/10
175/175 [==============================] - 1s 4ms/step - loss: 0.6453 - accuracy: 0.6264 - val_loss: 0.6568 - val_accuracy: 0.6700
Epoch 3/10
175/175 [==============================] - 1s 4ms/step - loss: 0.6298 - accuracy: 0.6461 - val_loss: 0.6409 - val_accuracy: 0.6586
Epoch 4/10
175/175 [==============================] - 1s 4ms/step - loss: 0.6264 - accuracy: 0.6459 - val_loss: 0.6249 - val_accuracy: 0.6693
Epoch 5/10
175/175 [==============================] - 1s 3ms/step - loss: 0.6183 - accuracy: 0.6579 - val_loss: 0.6106 - val_accuracy: 0.6764
Epoch 6/10
175/175 [==============================] - 1s 5ms/step - loss: 0.6108 - accuracy: 0.6691 - val_loss: 0.6012 - val_accuracy: 0.6850
Epoch 7/10
175/175 [============

In [ ]:
average_accuracy_ANN = np.mean(accuracy_scores_ANN)
average_f1_ANN = np.mean(f1_scores_ANN)
average_accuracy_LR = np.mean(accuracy_scores_LR)
average_f1_LR = np.mean(f1_scores_LR)
average_accuracy_KNN = np.mean(accuracy_scores_KNN)
average_f1_KNN = np.mean(f1_scores_KNN)
average_accuracy_SVM = np.mean(accuracy_scores_SVM)
average_f1_SVM = np.mean(f1_scores_SVM)
average_accuracy_RF = np.mean(accuracy_scores_RF)
average_f1_RF = np.mean(f1_scores_RF)
average_accuracy_XGB = np.mean(accuracy_scores_XGB)
average_f1_XGB = np.mean(f1_scores_XGB)

In [ ]:
print(accuracy_scores_ANN)

[0.614, 0.5613333333333334, 0.6806666666666666, 0.73, 0.5906666666666667]


In [ ]:
print(accuracy_scores_LR)

[0.588, 0.5673333333333334, 0.5786666666666667, 0.574, 0.586]


In [ ]:
print(accuracy_scores_KNN)

[0.602, 0.6086666666666667, 0.6053333333333333, 0.6073333333333333, 0.6006666666666667]


In [ ]:
print(accuracy_scores_SVM)

[0.5566666666666666, 0.5486666666666666, 0.55, 0.5533333333333333, 0.5566666666666666]


In [ ]:
print(accuracy_scores_RF)

[0.754, 0.7533333333333333, 0.7526666666666667, 0.7713333333333333, 0.7593333333333333]


In [ ]:
print(accuracy_scores_XGB)

[0.88, 0.8746666666666667, 0.8746666666666667, 0.88, 0.8746666666666667]


In [ ]:
print(f1_scores_ANN)

[0.7059421025901473, 0.23842592592592593, 0.7206997084548105, 0.731610337972167, 0.3590814196242171]


In [ ]:
print(f1_scores_LR)

[0.5896414342629482, 0.5863607393244105, 0.594871794871795, 0.5895953757225434, 0.5778382053025154]


In [ ]:
print(f1_scores_KNN)

[0.5952542372881355, 0.605775688381464, 0.598371777476255, 0.6049631120053655, 0.5955435516542876]


In [ ]:
print(f1_scores_SVM)

[0.573444515715202, 0.5874466788543571, 0.5794392523364486, 0.5822942643391521, 0.5684620376378974]


In [ ]:
print(f1_scores_RF)

[0.7674858223062382, 0.7690387016229713, 0.7626359564939219, 0.7833228048010108, 0.7707936507936508]


In [ ]:
print(f1_scores_XGB)

[0.8834196891191709, 0.8777633289986996, 0.877444589308996, 0.884020618556701, 0.8777633289986996]


In [ ]:
print("Average Accuracy for ANN model:", average_accuracy_ANN)

Average Accuracy for ANN model: 0.6353333333333333


In [ ]:
print("Average F1 for ANN model:", average_f1_ANN)

Average F1 for ANN model: 0.5511518989134535


In [ ]:
print("Average Accuracy for LR model:", average_accuracy_LR)

Average Accuracy for LR model: 0.5788


In [ ]:
print("Average F1 for LR model:", average_f1_LR)

Average F1 for LR model: 0.5876615098968425


In [ ]:
print("Average Accuracy for KNN model:", average_accuracy_KNN)

Average Accuracy for KNN model: 0.6047999999999999


In [ ]:
print("Average F1 for KNN model:", average_f1_KNN)

Average F1 for KNN model: 0.5999816733611015


In [ ]:
print("Average Accuracy for SVM model:", average_accuracy_SVM)

Average Accuracy for SVM model: 0.5530666666666667


In [ ]:
print("Average F1 for SVM model:", average_f1_SVM)

Average F1 for SVM model: 0.5782173497766114


In [ ]:
print("Average Accuracy for RF model:", average_accuracy_RF)

Average Accuracy for RF model: 0.7581333333333333


In [ ]:
print("Average F1 for RF model:", average_f1_RF)

Average F1 for RF model: 0.7706553872035585


In [ ]:
print("Average Accuracy for XGB model:", average_accuracy_XGB)

Average Accuracy for XGB model: 0.8768


In [ ]:
print("Average F1 for XGB model:", average_f1_XGB)

Average F1 for XGB model: 0.8800823109964535


###**Inception-V3**

Import all libraries required

In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score, confusion_matrix
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout
from keras.models import Sequential
import tensorflow as tf
import matplotlib.pyplot as plt
import cv2
from tensorflow.keras.layers import BatchNormalization
from keras.models import Model, Sequential
from keras.applications.xception import Xception
from keras.applications import *
import matplotlib.pyplot as plt
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline
from PIL import Image
import random
import os
import cv2
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from keras.callbacks import EarlyStopping
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm
from sklearn.decomposition import PCA

In [ ]:
data_dir = "colon_image_sets"
SIZE_X = SIZE_Y = 299

datagen = tf.keras.preprocessing.image.ImageDataGenerator(validation_split = 0.3)

train_it = datagen.flow_from_directory(data_dir,
                                       class_mode = "categorical",
                                       target_size = (SIZE_X,SIZE_Y),
                                       color_mode="rgb",
                                       batch_size = 12,
                                       shuffle = False,
                                       subset='training',
                                       seed = 42)

validate_it = datagen.flow_from_directory(data_dir,
                                       class_mode = "categorical",
                                       target_size = (SIZE_X, SIZE_Y),
                                       color_mode="rgb",
                                       batch_size = 12,
                                       shuffle = False,
                                       subset='validation',
                                       seed = 42)

Found 7000 images belonging to 2 classes.
Found 3000 images belonging to 2 classes.


In [ ]:
def get_features(base_model, train, validate):
    X_train = base_model.predict(train)
    y_train = train.classes

    X_val = base_model.predict(validate)
    y_val = validate.classes

    X_val, X_test, y_val, y_test = train_test_split(X_val, y_val, test_size = 0.5, shuffle = True)
    print('Shape of X_train----->', str(X_train.shape))
    print('Shape of X_val----->', str(X_val.shape))
    print('Shape of X_test----->', str(X_test.shape))
    return (X_train, X_val, X_test, y_train, y_val, y_test)

In [ ]:
def get_models():
    ANN = Sequential()
    ANN.add(Dense(128, input_dim = X_train.shape[1], activation = 'relu'))
    ANN.add(BatchNormalization())
    ANN.add(Dropout(0.2))
    ANN.add(Dense(64, activation='relu'))
    ANN.add(Dense(32, activation='relu'))
    ANN.add(Dense(16, activation='relu'))
    ANN.add(Dense(8, activation='relu'))
    ANN.add(Dense(len(train_it.class_indices), activation='softmax'))
    ANN.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    LR = LogisticRegression()

    KNN = KNeighborsClassifier(n_neighbors=50)

    SVM = SVC(kernel = 'linear')

    RF = RandomForestClassifier(n_estimators = 50)

    XGB = XGBClassifier(n_estimators = 50, use_label_encoder=False)

    return (ANN, LR, KNN, SVM, RF, XGB)

In [ ]:
def reshape_data(X_train, X_val, X_test):
    X_train = X_train.reshape(X_train.shape[0], -1)
    X_val = X_val.reshape(X_val.shape[0], -1)
    X_test = X_test.reshape(X_test.shape[0], -1)

    print("Shape after reshaping------->")
    print("X train------->", str(X_train.shape))
    print("X val-------->", str(X_val.shape))
    print("X test-------->", str(X_test.shape))

    return (X_train, X_val, X_test)

In [ ]:
def fit_ANN(model, X_train, y_train, X_val, y_test):
    es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)
    history = model.fit(X_train, y_train, validation_data=(X_val, y_test), epochs=10, verbose=1, callbacks=[es])
    return model

def fit_model(model, X_train, y_train):
    model.fit(X_train, y_train)
    return model

In [ ]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
base_model = InceptionV3(include_top=False, weights='imagenet', input_shape=(SIZE_X, SIZE_Y, 3))

for layer in base_model.layers:
    layer.trainable = False

model = Model(inputs=base_model.input, outputs=base_model.layers[-1].output)
model.summary()

Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_5 (InputLayer)           [(None, 299, 299, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_20 (Conv2D)             (None, 149, 149, 32  864         ['input_5[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization_1 (BatchNo  (None, 149, 149, 32  96         ['conv2d_20[0][0]']              
 rmalization)                   )                                                           

 conv2d_31 (Conv2D)             (None, 35, 35, 32)   6144        ['average_pooling2d[0][0]']      
                                                                                                  
 batch_normalization_6 (BatchNo  (None, 35, 35, 64)  192         ['conv2d_25[0][0]']              
 rmalization)                                                                                     
                                                                                                  
 batch_normalization_8 (BatchNo  (None, 35, 35, 64)  192         ['conv2d_27[0][0]']              
 rmalization)                                                                                     
                                                                                                  
 batch_normalization_11 (BatchN  (None, 35, 35, 96)  288         ['conv2d_30[0][0]']              
 ormalization)                                                                                    
          

                                                                                                  
 conv2d_42 (Conv2D)             (None, 35, 35, 64)   18432       ['mixed1[0][0]']                 
                                                                                                  
 batch_normalization_23 (BatchN  (None, 35, 35, 64)  192         ['conv2d_42[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activation_22 (Activation)     (None, 35, 35, 64)   0           ['batch_normalization_23[0][0]'] 
                                                                                                  
 conv2d_40 (Conv2D)             (None, 35, 35, 48)   13824       ['mixed1[0][0]']                 
                                                                                                  
 conv2d_43

                                                                                                  
 activation_29 (Activation)     (None, 17, 17, 96)   0           ['batch_normalization_30[0][0]'] 
                                                                                                  
 max_pooling2d_14 (MaxPooling2D  (None, 17, 17, 288)  0          ['mixed2[0][0]']                 
 )                                                                                                
                                                                                                  
 mixed3 (Concatenate)           (None, 17, 17, 768)  0           ['activation_26[0][0]',          
                                                                  'activation_29[0][0]',          
                                                                  'max_pooling2d_14[0][0]']       
                                                                                                  
 conv2d_54

 mixed4 (Concatenate)           (None, 17, 17, 768)  0           ['activation_30[0][0]',          
                                                                  'activation_33[0][0]',          
                                                                  'activation_38[0][0]',          
                                                                  'activation_39[0][0]']          
                                                                                                  
 conv2d_64 (Conv2D)             (None, 17, 17, 160)  122880      ['mixed4[0][0]']                 
                                                                                                  
 batch_normalization_45 (BatchN  (None, 17, 17, 160)  480        ['conv2d_64[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activatio

 conv2d_74 (Conv2D)             (None, 17, 17, 160)  122880      ['mixed5[0][0]']                 
                                                                                                  
 batch_normalization_55 (BatchN  (None, 17, 17, 160)  480        ['conv2d_74[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activation_54 (Activation)     (None, 17, 17, 160)  0           ['batch_normalization_55[0][0]'] 
                                                                                                  
 conv2d_75 (Conv2D)             (None, 17, 17, 160)  179200      ['activation_54[0][0]']          
                                                                                                  
 batch_normalization_56 (BatchN  (None, 17, 17, 160)  480        ['conv2d_75[0][0]']              
 ormalizat

 activation_64 (Activation)     (None, 17, 17, 192)  0           ['batch_normalization_65[0][0]'] 
                                                                                                  
 conv2d_85 (Conv2D)             (None, 17, 17, 192)  258048      ['activation_64[0][0]']          
                                                                                                  
 batch_normalization_66 (BatchN  (None, 17, 17, 192)  576        ['conv2d_85[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activation_65 (Activation)     (None, 17, 17, 192)  0           ['batch_normalization_66[0][0]'] 
                                                                                                  
 conv2d_81 (Conv2D)             (None, 17, 17, 192)  147456      ['mixed6[0][0]']                 
          

 ormalization)                                                                                    
                                                                                                  
 activation_73 (Activation)     (None, 17, 17, 192)  0           ['batch_normalization_74[0][0]'] 
                                                                                                  
 conv2d_90 (Conv2D)             (None, 17, 17, 192)  147456      ['mixed7[0][0]']                 
                                                                                                  
 conv2d_94 (Conv2D)             (None, 17, 17, 192)  258048      ['activation_73[0][0]']          
                                                                                                  
 batch_normalization_71 (BatchN  (None, 17, 17, 192)  576        ['conv2d_90[0][0]']              
 ormalization)                                                                                    
          

 ormalization)                                                                                    
                                                                                                  
 conv2d_104 (Conv2D)            (None, 8, 8, 192)    245760      ['average_pooling2d_7[0][0]']    
                                                                                                  
 batch_normalization_77 (BatchN  (None, 8, 8, 320)   960         ['conv2d_96[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activation_78 (Activation)     (None, 8, 8, 384)    0           ['batch_normalization_79[0][0]'] 
                                                                                                  
 activation_79 (Activation)     (None, 8, 8, 384)    0           ['batch_normalization_80[0][0]'] 
          

                                                                                                  
 activation_87 (Activation)     (None, 8, 8, 384)    0           ['batch_normalization_88[0][0]'] 
                                                                                                  
 activation_88 (Activation)     (None, 8, 8, 384)    0           ['batch_normalization_89[0][0]'] 
                                                                                                  
 activation_91 (Activation)     (None, 8, 8, 384)    0           ['batch_normalization_92[0][0]'] 
                                                                                                  
 activation_92 (Activation)     (None, 8, 8, 384)    0           ['batch_normalization_93[0][0]'] 
                                                                                                  
 batch_normalization_94 (BatchN  (None, 8, 8, 192)   576         ['conv2d_113[0][0]']             
 ormalizat

In [ ]:
X_train, X_val, X_test, y_train, y_val, y_test = get_features(model, train_it, validate_it)

Shape of X_train-----> (7000, 8, 8, 2048)
Shape of X_val-----> (1500, 8, 8, 2048)
Shape of X_test-----> (1500, 8, 8, 2048)


In [ ]:
X_train, X_val, X_test = reshape_data(X_train, X_val, X_test)
ANN, LR, KNN, SVM, RF, XGB = get_models()

from sklearn.model_selection import KFold

# Define number of splits for K-Fold cross-validation
k_folds = 5
kf = KFold(n_splits=k_folds, shuffle=True)

# Initialize lists to store performance metrics for each fold
accuracy_scores_ANN = []
f1_scores_ANN = []
accuracy_scores_LR = []
f1_scores_LR = []
accuracy_scores_KNN = []
f1_scores_KNN = []
accuracy_scores_SVM = []
f1_scores_SVM = []
accuracy_scores_RF = []
f1_scores_RF = []
accuracy_scores_XGB = []
f1_scores_XGB = []
# Perform K-Fold cross-validation
for train_index, val_index in kf.split(X_train):
    X_train_fold, X_val_fold = X_train[train_index], X_train[val_index]
    y_train_fold, y_val_fold = y_train[train_index], y_train[val_index]

    # Train ANN model
    ANN_fold = fit_ANN(ANN, X_train_fold, y_train_fold, X_val_fold, y_val_fold)
    # Evaluate ANN model
    accuracy_ANN = accuracy_score(y_test, np.argmax(ANN_fold.predict(X_test), axis=1))
    f1_ANN = f1_score(y_test, np.argmax(ANN_fold.predict(X_test), axis=1))
    accuracy_scores_ANN.append(accuracy_ANN)
    f1_scores_ANN.append(f1_ANN)

    # Train LR model
    LR_fold = fit_model(LR, X_train_fold, y_train_fold)
    # Evaluate LR model
    accuracy_LR = accuracy_score(y_test, LR_fold.predict(X_test))
    accuracy_scores_LR.append(accuracy_LR)
    f1_LR=f1_score(y_test, LR_fold.predict(X_test))
    f1_scores_LR.append(f1_LR)

    # Train KNN model
    KNN_fold = fit_model(KNN, X_train_fold, y_train_fold)
    # Evaluate KNN model
    accuracy_KNN = accuracy_score(y_test, KNN_fold.predict(X_test))
    accuracy_scores_KNN.append(accuracy_KNN)
    f1_KNN=f1_score(y_test, KNN_fold.predict(X_test))
    f1_scores_KNN.append(f1_KNN)

    # Train SVM model
    SVM_fold = fit_model(SVM, X_train_fold, y_train_fold)
    # Evaluate SVM model
    accuracy_SVM = accuracy_score(y_test, SVM_fold.predict(X_test))
    accuracy_scores_SVM.append(accuracy_SVM)
    f1_SVM=f1_score(y_test, SVM_fold.predict(X_test))
    f1_scores_SVM.append(f1_SVM)

    # Train RF model
    RF_fold = fit_model(RF, X_train_fold, y_train_fold)
    # Evaluate RF model
    accuracy_RF = accuracy_score(y_test, RF_fold.predict(X_test))
    accuracy_scores_RF.append(accuracy_RF)
    f1_RF=f1_score(y_test, RF_fold.predict(X_test))
    f1_scores_RF.append(f1_RF)

    # Train XGB model
    XGB_fold = fit_model(XGB, X_train_fold, y_train_fold)
    # Evaluate XGB model
    accuracy_XGB = accuracy_score(y_test, XGB_fold.predict(X_test))
    accuracy_scores_XGB.append(accuracy_XGB)
    f1_XGB=f1_score(y_test, XGB_fold.predict(X_test))
    f1_scores_XGB.append(f1_XGB)

Shape after reshaping------->
X train-------> (7000, 131072)
X val--------> (1500, 131072)
X test--------> (1500, 131072)
Epoch 1/10
175/175 [==============================] - 21s 117ms/step - loss: 0.3634 - accuracy: 0.8396 - val_loss: 1.1546 - val_accuracy: 0.6221
Epoch 2/10
175/175 [==============================] - 19s 107ms/step - loss: 0.1947 - accuracy: 0.9232 - val_loss: 0.2290 - val_accuracy: 0.9093
Epoch 3/10
175/175 [==============================] - 18s 103ms/step - loss: 0.1458 - accuracy: 0.9436 - val_loss: 0.3169 - val_accuracy: 0.8786
Epoch 4/10
175/175 [==============================] - 17s 99ms/step - loss: 0.1126 - accuracy: 0.9570 - val_loss: 0.1656 - val_accuracy: 0.9436
Epoch 5/10
175/175 [==============================] - 17s 97ms/step - loss: 0.1021 - accuracy: 0.9605 - val_loss: 0.2584 - val_accuracy: 0.8943
Epoch 6/10
175/175 [==============================] - 17s 99ms/step - loss: 0.0868 - accuracy: 0.9689 - val_loss: 0.1906 - val_accuracy: 0.9379
Epoch 7/10


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Epoch 1/10
175/175 [==============================] - 120s 687ms/step - loss: 0.0838 - accuracy: 0.9702 - val_loss: 0.0342 - val_accuracy: 0.9914
Epoch 2/10
175/175 [==============================] - 122s 697ms/step - loss: 0.0544 - accuracy: 0.9795 - val_loss: 0.1841 - val_accuracy: 0.9371
Epoch 3/10
175/175 [==============================] - 119s 684ms/step - loss: 0.0491 - accuracy: 0.9823 - val_loss: 0.1026 - val_accuracy: 0.9679
Epoch 4/10
175/175 [==============================] - 117s 668ms/step - loss: 0.0563 - accuracy: 0.9787 - val_loss: 1.8310 - val_accuracy: 0.5679
Epoch 5/10
175/175 [==============================] - 113s 646ms/step - loss: 0.0491 - accuracy: 0.9818 - val_loss: 0.0767 - val_accuracy: 0.9686
Epoch 6/10
175/175 [==============================] - 112s 640ms/step - loss: 0.0348 - accuracy: 0.9882 - val_loss: 0.0641 - val_accuracy: 0.9757
Epoch 6: early stopping


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Epoch 1/10
175/175 [==============================] - 76s 432ms/step - loss: 0.0636 - accuracy: 0.9757 - val_loss: 0.0595 - val_accuracy: 0.9850
Epoch 2/10
175/175 [==============================] - 76s 437ms/step - loss: 0.0309 - accuracy: 0.9887 - val_loss: 0.1075 - val_accuracy: 0.9957
Epoch 3/10
175/175 [==============================] - 76s 438ms/step - loss: 0.0387 - accuracy: 0.9854 - val_loss: 0.3715 - val_accuracy: 0.9707
Epoch 4/10
175/175 [==============================] - 77s 442ms/step - loss: 0.0225 - accuracy: 0.9911 - val_loss: 0.7611 - val_accuracy: 0.9786
Epoch 5/10
175/175 [==============================] - 75s 431ms/step - loss: 0.0413 - accuracy: 0.9846 - val_loss: 1.3947 - val_accuracy: 0.9757
Epoch 6/10
175/175 [==============================] - 74s 425ms/step - loss: 0.0355 - accuracy: 0.9870 - val_loss: 2.4501 - val_accuracy: 0.9693
Epoch 6: early stopping


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Epoch 1/10
175/175 [==============================] - 73s 418ms/step - loss: 0.0283 - accuracy: 0.9896 - val_loss: 0.0335 - val_accuracy: 0.9893
Epoch 2/10
175/175 [==============================] - 72s 413ms/step - loss: 0.0278 - accuracy: 0.9907 - val_loss: 0.1202 - val_accuracy: 0.9529
Epoch 3/10
175/175 [==============================] - 72s 411ms/step - loss: 0.0351 - accuracy: 0.9875 - val_loss: 0.0194 - val_accuracy: 0.9914
Epoch 4/10
175/175 [==============================] - 73s 416ms/step - loss: 0.0287 - accuracy: 0.9907 - val_loss: 0.0714 - val_accuracy: 0.9757
Epoch 5/10
175/175 [==============================] - 71s 407ms/step - loss: 0.0298 - accuracy: 0.9889 - val_loss: 0.0281 - val_accuracy: 0.9886
Epoch 6/10
175/175 [==============================] - 70s 402ms/step - loss: 0.0224 - accuracy: 0.9920 - val_loss: 1.0000 - val_accuracy: 0.7736
Epoch 7/10
175/175 [==============================] - 73s 417ms/step - loss: 0.0319 - accuracy: 0.9893 - val_loss: 0.0447 - val_ac

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Epoch 1/10
175/175 [==============================] - 69s 396ms/step - loss: 0.0325 - accuracy: 0.9887 - val_loss: 0.0191 - val_accuracy: 0.9943
Epoch 2/10
175/175 [==============================] - 71s 405ms/step - loss: 0.0224 - accuracy: 0.9921 - val_loss: 0.0027 - val_accuracy: 1.0000
Epoch 3/10
175/175 [==============================] - 71s 405ms/step - loss: 0.0239 - accuracy: 0.9909 - val_loss: 0.0566 - val_accuracy: 0.9800
Epoch 4/10
175/175 [==============================] - 70s 399ms/step - loss: 0.0260 - accuracy: 0.9925 - val_loss: 0.0911 - val_accuracy: 0.9600
Epoch 5/10
175/175 [==============================] - 69s 393ms/step - loss: 0.0250 - accuracy: 0.9914 - val_loss: 0.2702 - val_accuracy: 0.9143
Epoch 6/10
175/175 [==============================] - 69s 393ms/step - loss: 0.0139 - accuracy: 0.9952 - val_loss: 0.0490 - val_accuracy: 0.9793
Epoch 7/10
175/175 [==============================] - 69s 396ms/step - loss: 0.0223 - accuracy: 0.9912 - val_loss: 0.0143 - val_ac

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
average_accuracy_ANN = np.mean(accuracy_scores_ANN)
average_f1_ANN = np.mean(f1_scores_ANN)
average_accuracy_LR = np.mean(accuracy_scores_LR)
average_f1_LR = np.mean(f1_scores_LR)
average_accuracy_KNN = np.mean(accuracy_scores_KNN)
average_f1_KNN = np.mean(f1_scores_KNN)
average_accuracy_SVM = np.mean(accuracy_scores_SVM)
average_f1_SVM = np.mean(f1_scores_SVM)
average_accuracy_RF = np.mean(accuracy_scores_RF)
average_f1_RF = np.mean(f1_scores_RF)
average_accuracy_XGB = np.mean(accuracy_scores_XGB)
average_f1_XGB = np.mean(f1_scores_XGB)

In [ ]:
print(accuracy_scores_ANN)

[0.8706666666666667, 0.96, 0.9353333333333333, 0.9193333333333333, 0.9586666666666667]


In [ ]:
print(accuracy_scores_LR)

[0.9466666666666667, 0.9506666666666667, 0.9533333333333334, 0.9466666666666667, 0.9446666666666667]


In [ ]:
print(accuracy_scores_KNN)

[0.7366666666666667, 0.7373333333333333, 0.75, 0.738, 0.7346666666666667]


In [ ]:
print(accuracy_scores_SVM)

[0.9653333333333334, 0.9613333333333334, 0.9606666666666667, 0.9613333333333334, 0.9626666666666667]


In [ ]:
print(accuracy_scores_RF)

[0.9046666666666666, 0.8966666666666666, 0.8953333333333333, 0.9046666666666666, 0.8933333333333333]


In [ ]:
print(accuracy_scores_XGB)

[0.942, 0.9326666666666666, 0.924, 0.9413333333333334, 0.938]


In [ ]:
print(f1_scores_ANN)

[0.8835534213685473, 0.9589603283173734, 0.9362261669953977, 0.9242329367564184, 0.9581646423751687]


In [ ]:
print(f1_scores_LR)

[0.9462365591397849, 0.9507323568575233, 0.9531459170013385, 0.946164199192463, 0.9445557782231128]


In [ ]:
print(f1_scores_KNN)

[0.6457399103139013, 0.6469534050179211, 0.6696035242290749, 0.6494201605709188, 0.6420863309352518]


In [ ]:
print(f1_scores_SVM)

[0.9653794940079894, 0.9614873837981408, 0.9606404269513008, 0.961021505376344, 0.9628647214854111]


In [ ]:
print(f1_scores_RF)

[0.90473017988008, 0.8972829688535453, 0.895123580494322, 0.9044756179024716, 0.8941798941798942]


In [ ]:
print(f1_scores_XGB)

[0.9421157684630739, 0.9326217478318879, 0.9243027888446216, 0.9414114513981358, 0.93812375249501]


In [ ]:
print("Average Accuracy for ANN model:", average_accuracy_ANN)

Average Accuracy for ANN model: 0.9288000000000001


In [ ]:
print("Average F1 for ANN model:", average_f1_ANN)

Average F1 for ANN model: 0.9322274991625811


In [ ]:
print("Average Accuracy for LR model:", average_accuracy_LR)

Average Accuracy for LR model: 0.9484


In [ ]:
print("Average F1 for LR model:", average_f1_LR)

Average F1 for LR model: 0.9481669620828445


In [ ]:
print("Average Accuracy for KNN model:", average_accuracy_KNN)

Average Accuracy for KNN model: 0.7393333333333334


In [ ]:
print("Average F1 for KNN model:", average_f1_KNN)

Average F1 for KNN model: 0.6507606662134136


In [ ]:
print("Average Accuracy for SVM model:", average_accuracy_SVM)

Average Accuracy for SVM model: 0.9622666666666666


In [ ]:
print("Average F1 for SVM model:", average_f1_SVM)

Average F1 for SVM model: 0.9622787063238374


In [ ]:
print("Average Accuracy for RF model:", average_accuracy_RF)

Average Accuracy for RF model: 0.8989333333333333


In [ ]:
print("Average F1 for RF model:", average_f1_RF)

Average F1 for RF model: 0.8991584482620626


In [ ]:
print("Average Accuracy for XGB model:", average_accuracy_XGB)

Average Accuracy for XGB model: 0.9356


In [ ]:
print("Average F1 for XGB model:", average_f1_XGB)

Average F1 for XGB model: 0.9357151018065458


###**VGG-16**

Import all libraries required

In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score, confusion_matrix
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout
from keras.models import Sequential
import tensorflow as tf
import matplotlib.pyplot as plt
import cv2
from tensorflow.keras.layers import BatchNormalization
from keras.models import Model, Sequential
from keras.applications.xception import Xception
from keras.applications import *
import matplotlib.pyplot as plt
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline
from PIL import Image
import random
import os
import cv2
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from keras.callbacks import EarlyStopping
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm
from sklearn.decomposition import PCA

In [ ]:
data_dir = "colon_image_sets"
SIZE_X = SIZE_Y = 224

datagen = tf.keras.preprocessing.image.ImageDataGenerator(validation_split = 0.3)

train_it = datagen.flow_from_directory(data_dir,
                                       class_mode = "categorical",
                                       target_size = (SIZE_X,SIZE_Y),
                                       color_mode="rgb",
                                       batch_size = 12,
                                       shuffle = False,
                                       subset='training',
                                       seed = 42)

validate_it = datagen.flow_from_directory(data_dir,
                                       class_mode = "categorical",
                                       target_size = (SIZE_X, SIZE_Y),
                                       color_mode="rgb",
                                       batch_size = 12,
                                       shuffle = False,
                                       subset='validation',
                                       seed = 42)


Found 7000 images belonging to 2 classes.
Found 3000 images belonging to 2 classes.


In [ ]:
def get_features(base_model, train, validate):
    X_train = base_model.predict(train)
    y_train = train.classes

    X_val = base_model.predict(validate)
    y_val = validate.classes

    X_val, X_test, y_val, y_test = train_test_split(X_val, y_val, test_size = 0.5, shuffle = True)
    print('Shape of X_train----->', str(X_train.shape))
    print('Shape of X_val----->', str(X_val.shape))
    print('Shape of X_test----->', str(X_test.shape))
    return (X_train, X_val, X_test, y_train, y_val, y_test)

In [ ]:
def get_models():
    ANN = Sequential()
    ANN.add(Dense(128, input_dim = X_train.shape[1], activation = 'relu'))
    ANN.add(BatchNormalization())
    ANN.add(Dropout(0.2))
    ANN.add(Dense(64, activation='relu'))
    ANN.add(Dense(32, activation='relu'))
    ANN.add(Dense(16, activation='relu'))
    ANN.add(Dense(8, activation='relu'))
    ANN.add(Dense(len(train_it.class_indices), activation='softmax'))
    ANN.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    LR = LogisticRegression()

    KNN = KNeighborsClassifier(n_neighbors=50)

    SVM = SVC(kernel = 'linear')

    RF = RandomForestClassifier(n_estimators = 50)

    XGB = XGBClassifier(n_estimators = 50, use_label_encoder=False)

    return (ANN, LR, KNN, SVM, RF, XGB)

In [ ]:
def reshape_data(X_train, X_val, X_test):
    X_train = X_train.reshape(X_train.shape[0], -1)
    X_val = X_val.reshape(X_val.shape[0], -1)
    X_test = X_test.reshape(X_test.shape[0], -1)

    print("Shape after reshaping------->")
    print("X train------->", str(X_train.shape))
    print("X val-------->", str(X_val.shape))
    print("X test-------->", str(X_test.shape))

    return (X_train, X_val, X_test)

In [ ]:
def fit_ANN(model, X_train, y_train, X_val, y_test):
    es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)
    history = model.fit(X_train, y_train, validation_data=(X_val, y_test), epochs=10, verbose=1, callbacks=[es])
    return model

def fit_model(model, X_train, y_train):
    model.fit(X_train, y_train)
    return model

In [ ]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
base_model = VGG16(include_top=False, input_shape=(SIZE_X, SIZE_Y, 3), weights='imagenet')

for layer in base_model.layers:
    layer.trainable = False

model = Model(inputs=base_model.input, outputs=base_model.layers[-1].output)
model.summary()

Model: "model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   

In [ ]:
X_train, X_val, X_test, y_train, y_val, y_test = get_features(model, train_it, validate_it)

Shape of X_train-----> (7000, 7, 7, 512)
Shape of X_val-----> (1500, 7, 7, 512)
Shape of X_test-----> (1500, 7, 7, 512)


In [ ]:
X_train, X_val, X_test = reshape_data(X_train, X_val, X_test)
ANN, LR, KNN, SVM, RF, XGB = get_models()

from sklearn.model_selection import KFold

# Define number of splits for K-Fold cross-validation
k_folds = 5
kf = KFold(n_splits=k_folds, shuffle=True)

# Initialize lists to store performance metrics for each fold
accuracy_scores_ANN = []
f1_scores_ANN = []
accuracy_scores_LR = []
f1_scores_LR = []
accuracy_scores_KNN = []
f1_scores_KNN = []
accuracy_scores_SVM = []
f1_scores_SVM = []
accuracy_scores_RF = []
f1_scores_RF = []
accuracy_scores_XGB = []
f1_scores_XGB = []
# Perform K-Fold cross-validation
for train_index, val_index in kf.split(X_train):
    X_train_fold, X_val_fold = X_train[train_index], X_train[val_index]
    y_train_fold, y_val_fold = y_train[train_index], y_train[val_index]

    # Train ANN model
    ANN_fold = fit_ANN(ANN, X_train_fold, y_train_fold, X_val_fold, y_val_fold)
    # Evaluate ANN model
    accuracy_ANN = accuracy_score(y_test, np.argmax(ANN_fold.predict(X_test), axis=1))
    f1_ANN = f1_score(y_test, np.argmax(ANN_fold.predict(X_test), axis=1))
    accuracy_scores_ANN.append(accuracy_ANN)
    f1_scores_ANN.append(f1_ANN)

    # Train LR model
    LR_fold = fit_model(LR, X_train_fold, y_train_fold)
    # Evaluate LR model
    accuracy_LR = accuracy_score(y_test, LR_fold.predict(X_test))
    accuracy_scores_LR.append(accuracy_LR)
    f1_LR=f1_score(y_test, LR_fold.predict(X_test))
    f1_scores_LR.append(f1_LR)

    # Train KNN model
    KNN_fold = fit_model(KNN, X_train_fold, y_train_fold)
    # Evaluate KNN model
    accuracy_KNN = accuracy_score(y_test, KNN_fold.predict(X_test))
    accuracy_scores_KNN.append(accuracy_KNN)
    f1_KNN=f1_score(y_test, KNN_fold.predict(X_test))
    f1_scores_KNN.append(f1_KNN)

    # Train SVM model
    SVM_fold = fit_model(SVM, X_train_fold, y_train_fold)
    # Evaluate SVM model
    accuracy_SVM = accuracy_score(y_test, SVM_fold.predict(X_test))
    accuracy_scores_SVM.append(accuracy_SVM)
    f1_SVM=f1_score(y_test, SVM_fold.predict(X_test))
    f1_scores_SVM.append(f1_SVM)

    # Train RF model
    RF_fold = fit_model(RF, X_train_fold, y_train_fold)
    # Evaluate RF model
    accuracy_RF = accuracy_score(y_test, RF_fold.predict(X_test))
    accuracy_scores_RF.append(accuracy_RF)
    f1_RF=f1_score(y_test, RF_fold.predict(X_test))
    f1_scores_RF.append(f1_RF)

    # Train XGB model
    XGB_fold = fit_model(XGB, X_train_fold, y_train_fold)
    # Evaluate XGB model
    accuracy_XGB = accuracy_score(y_test, XGB_fold.predict(X_test))
    accuracy_scores_XGB.append(accuracy_XGB)
    f1_XGB=f1_score(y_test, XGB_fold.predict(X_test))
    f1_scores_XGB.append(f1_XGB)

Shape after reshaping------->
X train-------> (7000, 25088)
X val--------> (1500, 25088)
X test--------> (1500, 25088)
Epoch 1/10
175/175 [==============================] - 15s 83ms/step - loss: 0.0669 - accuracy: 0.9784 - val_loss: 0.0457 - val_accuracy: 0.9957
Epoch 2/10
175/175 [==============================] - 14s 83ms/step - loss: 0.0121 - accuracy: 0.9955 - val_loss: 0.0229 - val_accuracy: 0.9907
Epoch 3/10
175/175 [==============================] - 14s 81ms/step - loss: 0.0097 - accuracy: 0.9973 - val_loss: 0.0153 - val_accuracy: 0.9943
Epoch 4/10
175/175 [==============================] - 14s 81ms/step - loss: 0.0051 - accuracy: 0.9987 - val_loss: 0.0298 - val_accuracy: 0.9929
Epoch 5/10
175/175 [==============================] - 15s 83ms/step - loss: 0.0136 - accuracy: 0.9962 - val_loss: 0.0211 - val_accuracy: 0.9936
Epoch 6/10
175/175 [==============================] - 14s 80ms/step - loss: 0.0043 - accuracy: 0.9991 - val_loss: 0.0115 - val_accuracy: 0.9971
Epoch 7/10
175/17

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Epoch 1/10
175/175 [==============================] - 14s 83ms/step - loss: 0.0109 - accuracy: 0.9966 - val_loss: 0.0229 - val_accuracy: 0.9986
Epoch 2/10
175/175 [==============================] - 15s 84ms/step - loss: 0.0030 - accuracy: 0.9995 - val_loss: 0.0628 - val_accuracy: 0.9986
Epoch 3/10
175/175 [==============================] - 14s 80ms/step - loss: 5.7788e-04 - accuracy: 0.9998 - val_loss: 0.1215 - val_accuracy: 0.9986
Epoch 4/10
175/175 [==============================] - 14s 80ms/step - loss: 0.0036 - accuracy: 0.9991 - val_loss: 0.3544 - val_accuracy: 0.9979
Epoch 5/10
175/175 [==============================] - 14s 79ms/step - loss: 9.5337e-04 - accuracy: 0.9996 - val_loss: 0.3908 - val_accuracy: 0.9986
Epoch 6/10
175/175 [==============================] - 14s 80ms/step - loss: 0.0019 - accuracy: 0.9989 - val_loss: 0.5547 - val_accuracy: 0.9993
Epoch 6: early stopping


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Epoch 1/10
175/175 [==============================] - 14s 78ms/step - loss: 0.0039 - accuracy: 0.9986 - val_loss: 0.0027 - val_accuracy: 0.9993
Epoch 2/10
175/175 [==============================] - 14s 80ms/step - loss: 0.0022 - accuracy: 0.9991 - val_loss: 0.0032 - val_accuracy: 0.9986
Epoch 3/10
175/175 [==============================] - 14s 77ms/step - loss: 0.0052 - accuracy: 0.9980 - val_loss: 0.0016 - val_accuracy: 0.9993
Epoch 4/10
175/175 [==============================] - 14s 79ms/step - loss: 0.0066 - accuracy: 0.9982 - val_loss: 0.0155 - val_accuracy: 0.9979
Epoch 5/10
175/175 [==============================] - 14s 80ms/step - loss: 0.0053 - accuracy: 0.9982 - val_loss: 0.0328 - val_accuracy: 0.9986
Epoch 6/10
175/175 [==============================] - 13s 77ms/step - loss: 0.0051 - accuracy: 0.9986 - val_loss: 0.0168 - val_accuracy: 0.9993
Epoch 7/10
175/175 [==============================] - 13s 75ms/step - loss: 0.0033 - accuracy: 0.9987 - val_loss: 0.0279 - val_accuracy:

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Epoch 1/10
175/175 [==============================] - 14s 81ms/step - loss: 0.0018 - accuracy: 0.9998 - val_loss: 0.0543 - val_accuracy: 0.9971
Epoch 2/10
175/175 [==============================] - 15s 83ms/step - loss: 0.0052 - accuracy: 0.9986 - val_loss: 0.0591 - val_accuracy: 0.9971
Epoch 3/10
175/175 [==============================] - 14s 83ms/step - loss: 0.0018 - accuracy: 0.9995 - val_loss: 0.1967 - val_accuracy: 0.9971
Epoch 4/10
175/175 [==============================] - 14s 83ms/step - loss: 0.0036 - accuracy: 0.9986 - val_loss: 0.1479 - val_accuracy: 0.9971
Epoch 5/10
175/175 [==============================] - 14s 82ms/step - loss: 0.0015 - accuracy: 0.9993 - val_loss: 0.0539 - val_accuracy: 0.9986
Epoch 6/10
175/175 [==============================] - 15s 86ms/step - loss: 0.0021 - accuracy: 0.9993 - val_loss: 0.0123 - val_accuracy: 0.9993
Epoch 7/10
175/175 [==============================] - 15s 83ms/step - loss: 0.0028 - accuracy: 0.9991 - val_loss: 0.0156 - val_accuracy:

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Epoch 1/10
175/175 [==============================] - 15s 84ms/step - loss: 0.0025 - accuracy: 0.9991 - val_loss: 0.0198 - val_accuracy: 0.9986
Epoch 2/10
175/175 [==============================] - 13s 75ms/step - loss: 0.0034 - accuracy: 0.9991 - val_loss: 0.0467 - val_accuracy: 0.9979
Epoch 3/10
175/175 [==============================] - 13s 76ms/step - loss: 0.0010 - accuracy: 0.9996 - val_loss: 0.1497 - val_accuracy: 0.9971
Epoch 4/10
175/175 [==============================] - 14s 79ms/step - loss: 0.0018 - accuracy: 0.9993 - val_loss: 0.4039 - val_accuracy: 0.9964
Epoch 5/10
175/175 [==============================] - 14s 80ms/step - loss: 3.1033e-04 - accuracy: 1.0000 - val_loss: 0.7240 - val_accuracy: 0.9964
Epoch 6/10
175/175 [==============================] - 14s 79ms/step - loss: 7.2328e-04 - accuracy: 0.9998 - val_loss: 0.9061 - val_accuracy: 0.9964
Epoch 6: early stopping


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
average_accuracy_ANN = np.mean(accuracy_scores_ANN)
average_f1_ANN = np.mean(f1_scores_ANN)
average_accuracy_LR = np.mean(accuracy_scores_LR)
average_f1_LR = np.mean(f1_scores_LR)
average_accuracy_KNN = np.mean(accuracy_scores_KNN)
average_f1_KNN = np.mean(f1_scores_KNN)
average_accuracy_SVM = np.mean(accuracy_scores_SVM)
average_f1_SVM = np.mean(f1_scores_SVM)
average_accuracy_RF = np.mean(accuracy_scores_RF)
average_f1_RF = np.mean(f1_scores_RF)
average_accuracy_XGB = np.mean(accuracy_scores_XGB)
average_f1_XGB = np.mean(f1_scores_XGB)

In [ ]:
print(accuracy_scores_ANN)

[0.9946666666666667, 0.9966666666666667, 0.9946666666666667, 0.9966666666666667, 0.9973333333333333]


In [ ]:
print(accuracy_scores_LR)

[0.9946666666666667, 0.9926666666666667, 0.9946666666666667, 0.9946666666666667, 0.9953333333333333]


In [ ]:
print(accuracy_scores_KNN)

[0.612, 0.6293333333333333, 0.618, 0.616, 0.6093333333333333]


In [ ]:
print(accuracy_scores_SVM)

[0.9953333333333333, 0.9926666666666667, 0.9946666666666667, 0.9926666666666667, 0.9946666666666667]


In [ ]:
print(accuracy_scores_RF)

[0.982, 0.9833333333333333, 0.9833333333333333, 0.9873333333333333, 0.9833333333333333]


In [ ]:
print(accuracy_scores_XGB)

[0.9853333333333333, 0.9853333333333333, 0.986, 0.9873333333333333, 0.9866666666666667]


In [ ]:
print(f1_scores_ANN)

[0.9946018893387315, 0.9966329966329965, 0.9946018893387315, 0.9966284558327714, 0.9973082099596231]


In [ ]:
print(f1_scores_LR)

[0.9946091644204853, 0.9925925925925926, 0.9946091644204853, 0.9946164199192463, 0.9952861952861952]


In [ ]:
print(f1_scores_KNN)

[0.3561946902654867, 0.4021505376344086, 0.3723986856516977, 0.36703296703296706, 0.3488888888888889]


In [ ]:
print(f1_scores_SVM)

[0.99527983816588, 0.9925826028320971, 0.9946018893387315, 0.9925925925925926, 0.9946091644204853]


In [ ]:
print(f1_scores_RF)

[0.9817444219066935, 0.9830966869506425, 0.9830278343516633, 0.9871708305199188, 0.9830508474576272]


In [ ]:
print(f1_scores_XGB)

[0.9851150202976996, 0.9851150202976996, 0.9858203916272789, 0.987136086662153, 0.9864314789687924]


In [ ]:
print("Average Accuracy for ANN model:", average_accuracy_ANN)

Average Accuracy for ANN model: 0.9960000000000001


In [ ]:
print("Average F1 for ANN model:", average_f1_ANN)

Average F1 for ANN model: 0.9959546882205708


In [ ]:
print("Average Accuracy for LR model:", average_accuracy_LR)

Average Accuracy for LR model: 0.9944000000000001


In [ ]:
print("Average F1 for LR model:", average_f1_LR)

Average F1 for LR model: 0.994342707327801


In [ ]:
print("Average Accuracy for KNN model:", average_accuracy_KNN)

Average Accuracy for KNN model: 0.6169333333333333


In [ ]:
print("Average F1 for KNN model:", average_f1_KNN)

Average F1 for KNN model: 0.36933315389468985


In [ ]:
print("Average Accuracy for SVM model:", average_accuracy_SVM)

Average Accuracy for SVM model: 0.994


In [ ]:
print("Average F1 for SVM model:", average_f1_SVM)

Average F1 for SVM model: 0.9939332174699572


In [ ]:
print("Average Accuracy for RF model:", average_accuracy_RF)

Average Accuracy for RF model: 0.9838666666666667


In [ ]:
print("Average F1 for RF model:", average_f1_RF)

Average F1 for RF model: 0.983618124237309


In [ ]:
print("Average Accuracy for XGB model:", average_accuracy_XGB)

Average Accuracy for XGB model: 0.9861333333333333


In [ ]:
print("Average F1 for XGB model:", average_f1_XGB)

Average F1 for XGB model: 0.9859235995707246


###**ResNet-50**

Import all libraries required

In [ ]:

import numpy as np
import pandas as pd
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score, confusion_matrix
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout
from keras.models import Sequential
import tensorflow as tf
import matplotlib.pyplot as plt
import cv2
from tensorflow.keras.layers import BatchNormalization
from keras.models import Model, Sequential
from keras.applications.xception import Xception
from keras.applications import *
import matplotlib.pyplot as plt
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline
from PIL import Image
import random
import os
import cv2
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from keras.callbacks import EarlyStopping
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm
from sklearn.decomposition import PCA

In [ ]:
data_dir = "colon_image_sets"
SIZE_X = SIZE_Y = 224

datagen = tf.keras.preprocessing.image.ImageDataGenerator(validation_split = 0.3)

train_it = datagen.flow_from_directory(data_dir,
                                       class_mode = "categorical",
                                       target_size = (SIZE_X,SIZE_Y),
                                       color_mode="rgb",
                                       batch_size = 12,
                                       shuffle = False,
                                       subset='training',
                                       seed = 42)

validate_it = datagen.flow_from_directory(data_dir,
                                       class_mode = "categorical",
                                       target_size = (SIZE_X, SIZE_Y),
                                       color_mode="rgb",
                                       batch_size = 12,
                                       shuffle = False,
                                       subset='validation',
                                       seed = 42)

Found 7000 images belonging to 2 classes.
Found 3000 images belonging to 2 classes.


In [ ]:
def get_features(base_model, train, validate):
    X_train = base_model.predict(train)
    y_train = train.classes

    X_val = base_model.predict(validate)
    y_val = validate.classes

    X_val, X_test, y_val, y_test = train_test_split(X_val, y_val, test_size = 0.5, shuffle = True)
    print('Shape of X_train----->', str(X_train.shape))
    print('Shape of X_val----->', str(X_val.shape))
    print('Shape of X_test----->', str(X_test.shape))
    return (X_train, X_val, X_test, y_train, y_val, y_test)

In [ ]:
def get_models():
    ANN = Sequential()
    ANN.add(Dense(128, input_dim = X_train.shape[1], activation = 'relu'))
    ANN.add(BatchNormalization())
    ANN.add(Dropout(0.2))
    ANN.add(Dense(64, activation='relu'))
    ANN.add(Dense(32, activation='relu'))
    ANN.add(Dense(16, activation='relu'))
    ANN.add(Dense(8, activation='relu'))
    ANN.add(Dense(len(train_it.class_indices), activation='softmax'))
    ANN.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    LR = LogisticRegression()

    KNN = KNeighborsClassifier(n_neighbors=50)

    SVM = SVC(kernel = 'linear')

    RF = RandomForestClassifier(n_estimators = 50)

    XGB = XGBClassifier(n_estimators = 50, use_label_encoder=False)

    return (ANN, LR, KNN, SVM, RF, XGB)

In [ ]:
def reshape_data(X_train, X_val, X_test):
    X_train = X_train.reshape(X_train.shape[0], -1)
    X_val = X_val.reshape(X_val.shape[0], -1)
    X_test = X_test.reshape(X_test.shape[0], -1)

    print("Shape after reshaping------->")
    print("X train------->", str(X_train.shape))
    print("X val-------->", str(X_val.shape))
    print("X test-------->", str(X_test.shape))

    return (X_train, X_val, X_test)

In [ ]:
def fit_ANN(model, X_train, y_train, X_val, y_test):
    es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)
    history = model.fit(X_train, y_train, validation_data=(X_val, y_test), epochs=10, verbose=1, callbacks=[es])
    return model

def fit_model(model, X_train, y_train):
    model.fit(X_train, y_train)
    return model

In [ ]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model

base_model = ResNet50(include_top=False, weights='imagenet', input_shape=(SIZE_X, SIZE_Y, 3))

for layer in base_model.layers:
    layer.trainable = False

model = Model(inputs=base_model.input, outputs=base_model.layers[-1].output)
model.summary()

Model: "model_6"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_7 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_7[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                            

                                                                                                  
 conv2_block3_1_relu (Activatio  (None, 56, 56, 64)  0           ['conv2_block3_1_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv2_block3_2_conv (Conv2D)   (None, 56, 56, 64)   36928       ['conv2_block3_1_relu[0][0]']    
                                                                                                  
 conv2_block3_2_bn (BatchNormal  (None, 56, 56, 64)  256         ['conv2_block3_2_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv2_block3_2_relu (Activatio  (None, 56, 56, 64)  0           ['conv2_block3_2_bn[0][0]']      
 n)       

                                                                                                  
 conv3_block3_1_relu (Activatio  (None, 28, 28, 128)  0          ['conv3_block3_1_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv3_block3_2_conv (Conv2D)   (None, 28, 28, 128)  147584      ['conv3_block3_1_relu[0][0]']    
                                                                                                  
 conv3_block3_2_bn (BatchNormal  (None, 28, 28, 128)  512        ['conv3_block3_2_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv3_block3_2_relu (Activatio  (None, 28, 28, 128)  0          ['conv3_block3_2_bn[0][0]']      
 n)       

                                                                                                  
 conv4_block2_1_bn (BatchNormal  (None, 14, 14, 256)  1024       ['conv4_block2_1_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv4_block2_1_relu (Activatio  (None, 14, 14, 256)  0          ['conv4_block2_1_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv4_block2_2_conv (Conv2D)   (None, 14, 14, 256)  590080      ['conv4_block2_1_relu[0][0]']    
                                                                                                  
 conv4_block2_2_bn (BatchNormal  (None, 14, 14, 256)  1024       ['conv4_block2_2_conv[0][0]']    
 ization) 

 conv4_block5_1_conv (Conv2D)   (None, 14, 14, 256)  262400      ['conv4_block4_out[0][0]']       
                                                                                                  
 conv4_block5_1_bn (BatchNormal  (None, 14, 14, 256)  1024       ['conv4_block5_1_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv4_block5_1_relu (Activatio  (None, 14, 14, 256)  0          ['conv4_block5_1_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv4_block5_2_conv (Conv2D)   (None, 14, 14, 256)  590080      ['conv4_block5_1_relu[0][0]']    
                                                                                                  
 conv4_blo

                                                                  'conv5_block1_3_bn[0][0]']      
                                                                                                  
 conv5_block1_out (Activation)  (None, 7, 7, 2048)   0           ['conv5_block1_add[0][0]']       
                                                                                                  
 conv5_block2_1_conv (Conv2D)   (None, 7, 7, 512)    1049088     ['conv5_block1_out[0][0]']       
                                                                                                  
 conv5_block2_1_bn (BatchNormal  (None, 7, 7, 512)   2048        ['conv5_block2_1_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv5_block2_1_relu (Activatio  (None, 7, 7, 512)   0           ['conv5_block2_1_bn[0][0]']      
 n)       

In [ ]:
X_train, X_val, X_test, y_train, y_val, y_test = get_features(model, train_it, validate_it)

Shape of X_train-----> (7000, 7, 7, 2048)
Shape of X_val-----> (1500, 7, 7, 2048)
Shape of X_test-----> (1500, 7, 7, 2048)


In [ ]:
X_train, X_val, X_test = reshape_data(X_train, X_val, X_test)
ANN, LR, KNN, SVM, RF, XGB = get_models()

from sklearn.model_selection import KFold

# Define number of splits for K-Fold cross-validation
k_folds = 5
kf = KFold(n_splits=k_folds, shuffle=True)

# Initialize lists to store performance metrics for each fold
accuracy_scores_ANN = []
f1_scores_ANN = []
accuracy_scores_LR = []
f1_scores_LR = []
accuracy_scores_KNN = []
f1_scores_KNN = []
accuracy_scores_SVM = []
f1_scores_SVM = []
accuracy_scores_RF = []
f1_scores_RF = []
accuracy_scores_XGB = []
f1_scores_XGB = []
# Perform K-Fold cross-validation
for train_index, val_index in kf.split(X_train):
    X_train_fold, X_val_fold = X_train[train_index], X_train[val_index]
    y_train_fold, y_val_fold = y_train[train_index], y_train[val_index]

    # Train ANN model
    ANN_fold = fit_ANN(ANN, X_train_fold, y_train_fold, X_val_fold, y_val_fold)
    # Evaluate ANN model
    accuracy_ANN = accuracy_score(y_test, np.argmax(ANN_fold.predict(X_test), axis=1))
    f1_ANN = f1_score(y_test, np.argmax(ANN_fold.predict(X_test), axis=1))
    accuracy_scores_ANN.append(accuracy_ANN)
    f1_scores_ANN.append(f1_ANN)

    # Train LR model
    LR_fold = fit_model(LR, X_train_fold, y_train_fold)
    # Evaluate LR model
    accuracy_LR = accuracy_score(y_test, LR_fold.predict(X_test))
    accuracy_scores_LR.append(accuracy_LR)
    f1_LR=f1_score(y_test, LR_fold.predict(X_test))
    f1_scores_LR.append(f1_LR)

    # Train KNN model
    KNN_fold = fit_model(KNN, X_train_fold, y_train_fold)
    # Evaluate KNN model
    accuracy_KNN = accuracy_score(y_test, KNN_fold.predict(X_test))
    accuracy_scores_KNN.append(accuracy_KNN)
    f1_KNN=f1_score(y_test, KNN_fold.predict(X_test))
    f1_scores_KNN.append(f1_KNN)

    # Train SVM model
    SVM_fold = fit_model(SVM, X_train_fold, y_train_fold)
    # Evaluate SVM model
    accuracy_SVM = accuracy_score(y_test, SVM_fold.predict(X_test))
    accuracy_scores_SVM.append(accuracy_SVM)
    f1_SVM=f1_score(y_test, SVM_fold.predict(X_test))
    f1_scores_SVM.append(f1_SVM)

    # Train RF model
    RF_fold = fit_model(RF, X_train_fold, y_train_fold)
    # Evaluate RF model
    accuracy_RF = accuracy_score(y_test, RF_fold.predict(X_test))
    accuracy_scores_RF.append(accuracy_RF)
    f1_RF=f1_score(y_test, RF_fold.predict(X_test))
    f1_scores_RF.append(f1_RF)

    # Train XGB model
    XGB_fold = fit_model(XGB, X_train_fold, y_train_fold)
    # Evaluate XGB model
    accuracy_XGB = accuracy_score(y_test, XGB_fold.predict(X_test))
    accuracy_scores_XGB.append(accuracy_XGB)
    f1_XGB=f1_score(y_test, XGB_fold.predict(X_test))
    f1_scores_XGB.append(f1_XGB)

Shape after reshaping------->
X train-------> (7000, 100352)
X val--------> (1500, 100352)
X test--------> (1500, 100352)
Epoch 1/10
175/175 [==============================] - 53s 300ms/step - loss: 0.0559 - accuracy: 0.9804 - val_loss: 0.0434 - val_accuracy: 0.9821
Epoch 2/10
175/175 [==============================] - 49s 281ms/step - loss: 0.0111 - accuracy: 0.9968 - val_loss: 0.0147 - val_accuracy: 0.9936
Epoch 3/10
175/175 [==============================] - 50s 285ms/step - loss: 0.0068 - accuracy: 0.9980 - val_loss: 0.0204 - val_accuracy: 0.9936
Epoch 4/10
175/175 [==============================] - 50s 288ms/step - loss: 0.0099 - accuracy: 0.9964 - val_loss: 0.0162 - val_accuracy: 0.9950
Epoch 5/10
175/175 [==============================] - 50s 287ms/step - loss: 0.0041 - accuracy: 0.9986 - val_loss: 0.0069 - val_accuracy: 0.9957
Epoch 6/10
175/175 [==============================] - 50s 286ms/step - loss: 0.0040 - accuracy: 0.9986 - val_loss: 0.0131 - val_accuracy: 0.9979
Epoch 7/

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Epoch 1/10
175/175 [==============================] - 39s 224ms/step - loss: 0.0086 - accuracy: 0.9962 - val_loss: 0.0309 - val_accuracy: 0.9986
Epoch 2/10
175/175 [==============================] - 40s 226ms/step - loss: 0.0026 - accuracy: 0.9989 - val_loss: 0.0885 - val_accuracy: 0.9986
Epoch 3/10
175/175 [==============================] - 40s 227ms/step - loss: 7.5287e-04 - accuracy: 0.9998 - val_loss: 0.2897 - val_accuracy: 0.9986
Epoch 4/10
175/175 [==============================] - 40s 227ms/step - loss: 0.0036 - accuracy: 0.9987 - val_loss: 0.3987 - val_accuracy: 0.9979
Epoch 5/10
175/175 [==============================] - 39s 224ms/step - loss: 0.0074 - accuracy: 0.9980 - val_loss: 0.1798 - val_accuracy: 0.9986
Epoch 6/10
175/175 [==============================] - 40s 231ms/step - loss: 0.0019 - accuracy: 0.9996 - val_loss: 0.1902 - val_accuracy: 0.9986
Epoch 6: early stopping


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Epoch 1/10
175/175 [==============================] - 40s 231ms/step - loss: 0.0035 - accuracy: 0.9987 - val_loss: 0.0059 - val_accuracy: 0.9986
Epoch 2/10
175/175 [==============================] - 40s 227ms/step - loss: 0.0030 - accuracy: 0.9991 - val_loss: 0.0538 - val_accuracy: 0.9986
Epoch 3/10
175/175 [==============================] - 41s 237ms/step - loss: 0.0037 - accuracy: 0.9991 - val_loss: 0.0720 - val_accuracy: 0.9993
Epoch 4/10
175/175 [==============================] - 40s 228ms/step - loss: 0.0027 - accuracy: 0.9987 - val_loss: 0.2105 - val_accuracy: 0.9993
Epoch 5/10
175/175 [==============================] - 40s 232ms/step - loss: 0.0026 - accuracy: 0.9991 - val_loss: 0.4183 - val_accuracy: 0.9993
Epoch 6/10
175/175 [==============================] - 41s 234ms/step - loss: 8.4341e-04 - accuracy: 0.9998 - val_loss: 0.3504 - val_accuracy: 0.9993
Epoch 6: early stopping


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Epoch 1/10
175/175 [==============================] - 41s 234ms/step - loss: 7.0007e-05 - accuracy: 1.0000 - val_loss: 1.1115e-06 - val_accuracy: 1.0000
Epoch 2/10
175/175 [==============================] - 41s 236ms/step - loss: 5.3308e-04 - accuracy: 0.9998 - val_loss: 7.4211e-05 - val_accuracy: 1.0000
Epoch 3/10
175/175 [==============================] - 42s 239ms/step - loss: 0.0029 - accuracy: 0.9987 - val_loss: 0.0011 - val_accuracy: 0.9993
Epoch 4/10
175/175 [==============================] - 42s 239ms/step - loss: 0.0039 - accuracy: 0.9986 - val_loss: 3.6099e-04 - val_accuracy: 1.0000
Epoch 5/10
175/175 [==============================] - 41s 237ms/step - loss: 0.0028 - accuracy: 0.9989 - val_loss: 2.4971e-05 - val_accuracy: 1.0000
Epoch 6/10
175/175 [==============================] - 41s 236ms/step - loss: 3.8070e-04 - accuracy: 0.9998 - val_loss: 1.1771e-04 - val_accuracy: 1.0000
Epoch 6: early stopping


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Epoch 1/10
175/175 [==============================] - 42s 240ms/step - loss: 0.0015 - accuracy: 0.9996 - val_loss: 0.0417 - val_accuracy: 0.9993
Epoch 2/10
175/175 [==============================] - 41s 236ms/step - loss: 1.8969e-04 - accuracy: 1.0000 - val_loss: 0.0755 - val_accuracy: 0.9993
Epoch 3/10
175/175 [==============================] - 42s 238ms/step - loss: 0.0019 - accuracy: 0.9998 - val_loss: 0.0676 - val_accuracy: 0.9993
Epoch 4/10
175/175 [==============================] - 41s 232ms/step - loss: 0.0027 - accuracy: 0.9987 - val_loss: 0.0146 - val_accuracy: 0.9993
Epoch 5/10
175/175 [==============================] - 41s 234ms/step - loss: 0.0038 - accuracy: 0.9986 - val_loss: 0.0809 - val_accuracy: 0.9993
Epoch 6/10
175/175 [==============================] - 41s 237ms/step - loss: 0.0044 - accuracy: 0.9977 - val_loss: 8.1696e-04 - val_accuracy: 0.9993
Epoch 7/10
175/175 [==============================] - 43s 245ms/step - loss: 0.0038 - accuracy: 0.9986 - val_loss: 4.3257e

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
average_accuracy_ANN = np.mean(accuracy_scores_ANN)
average_f1_ANN = np.mean(f1_scores_ANN)
average_accuracy_LR = np.mean(accuracy_scores_LR)
average_f1_LR = np.mean(f1_scores_LR)
average_accuracy_KNN = np.mean(accuracy_scores_KNN)
average_f1_KNN = np.mean(f1_scores_KNN)
average_accuracy_SVM = np.mean(accuracy_scores_SVM)
average_f1_SVM = np.mean(f1_scores_SVM)
average_accuracy_RF = np.mean(accuracy_scores_RF)
average_f1_RF = np.mean(f1_scores_RF)
average_accuracy_XGB = np.mean(accuracy_scores_XGB)
average_f1_XGB = np.mean(f1_scores_XGB)

In [ ]:
print(accuracy_scores_ANN)

[0.9986666666666667, 0.998, 0.998, 0.9993333333333333, 0.9986666666666667]


In [ ]:
print(accuracy_scores_LR)

[0.9993333333333333, 0.9993333333333333, 0.998, 0.9986666666666667, 0.9986666666666667]


In [ ]:
print(accuracy_scores_KNN)

[0.8833333333333333, 0.89, 0.8886666666666667, 0.8846666666666667, 0.8846666666666667]


In [ ]:
print(accuracy_scores_SVM)

[0.9986666666666667, 0.9986666666666667, 0.9986666666666667, 0.9986666666666667, 0.9986666666666667]


In [ ]:
print(accuracy_scores_RF)

[0.9846666666666667, 0.982, 0.9793333333333333, 0.984, 0.9826666666666667]


In [ ]:
print(accuracy_scores_XGB)

[0.9926666666666667, 0.9886666666666667, 0.9906666666666667, 0.9826666666666667, 0.9913333333333333]


In [ ]:
print(f1_scores_ANN)

[0.9986522911051213, 0.997979797979798, 0.9979825151311366, 0.9993265993265993, 0.9986541049798116]


In [ ]:
print(f1_scores_LR)

[0.9993275050437123, 0.9993275050437123, 0.9979852249832102, 0.9986541049798116, 0.9986541049798116]


In [ ]:
print(f1_scores_KNN)

[0.86651411136537, 0.8750946252838759, 0.8733889310083396, 0.8682406702208683, 0.8682406702208683]


In [ ]:
print(f1_scores_SVM)

[0.9986541049798116, 0.9986541049798116, 0.9986559139784946, 0.9986541049798116, 0.9986541049798116]


In [ ]:
print(f1_scores_RF)

[0.9845117845117846, 0.9818181818181818, 0.9790964261631827, 0.9838274932614555, 0.9824561403508772]


In [ ]:
print(f1_scores_XGB)

[0.9926124916051041, 0.98859825620389, 0.9906040268456376, 0.9825503355704698, 0.9912457912457913]


In [ ]:
print("Average Accuracy for ANN model:", average_accuracy_ANN)

Average Accuracy for ANN model: 0.9985333333333333


In [ ]:
print("Average F1 for ANN model:", average_f1_ANN)

Average F1 for ANN model: 0.9985190617044933


In [ ]:
print("Average Accuracy for LR model:", average_accuracy_LR)

Average Accuracy for LR model: 0.9987999999999999


In [ ]:
print("Average F1 for LR model:", average_f1_LR)

Average F1 for LR model: 0.9987896890060515


In [ ]:
print("Average Accuracy for KNN model:", average_accuracy_KNN)

Average Accuracy for KNN model: 0.8862666666666668


In [ ]:
print("Average F1 for KNN model:", average_f1_KNN)


Average F1 for KNN model: 0.8702958016198643


In [ ]:
print("Average Accuracy for SVM model:", average_accuracy_SVM)

Average Accuracy for SVM model: 0.9986666666666666


In [ ]:
print("Average F1 for SVM model:", average_f1_SVM)

Average F1 for SVM model: 0.9986544667795482


In [ ]:
print("Average Accuracy for RF model:", average_accuracy_RF)

Average Accuracy for RF model: 0.9825333333333333


In [ ]:
print("Average F1 for RF model:", average_f1_RF)

Average F1 for RF model: 0.9823420052210963


In [ ]:
print("Average Accuracy for XGB model:", average_accuracy_XGB)


Average Accuracy for XGB model: 0.9892


In [ ]:
print("Average F1 for XGB model:", average_f1_XGB)

Average F1 for XGB model: 0.9891221802941785


###**DenseNet**

Import all libraries required

In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score, confusion_matrix
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout
from keras.models import Sequential
import tensorflow as tf
import matplotlib.pyplot as plt
import cv2
from tensorflow.keras.layers import BatchNormalization
from keras.models import Model, Sequential
from keras.applications.xception import Xception
from keras.applications import *
import matplotlib.pyplot as plt
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline
from PIL import Image
import random
import os
import cv2
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from keras.callbacks import EarlyStopping
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm
from sklearn.decomposition import PCA

In [ ]:
data_dir = "colon_image_sets"
SIZE_X = SIZE_Y = 224

datagen = tf.keras.preprocessing.image.ImageDataGenerator(validation_split = 0.3)

train_it = datagen.flow_from_directory(data_dir,
                                       class_mode = "categorical",
                                       target_size = (SIZE_X,SIZE_Y),
                                       color_mode="rgb",
                                       batch_size = 12,
                                       shuffle = False,
                                       subset='training',
                                       seed = 42)

validate_it = datagen.flow_from_directory(data_dir,
                                       class_mode = "categorical",
                                       target_size = (SIZE_X, SIZE_Y),
                                       color_mode="rgb",
                                       batch_size = 12,
                                       shuffle = False,
                                       subset='validation',
                                       seed = 42)

Found 7000 images belonging to 2 classes.
Found 3000 images belonging to 2 classes.


In [ ]:
def get_features(base_model, train, validate):
    X_train = base_model.predict(train)
    y_train = train.classes

    X_val = base_model.predict(validate)
    y_val = validate.classes

    X_val, X_test, y_val, y_test = train_test_split(X_val, y_val, test_size = 0.5, shuffle = True)
    print('Shape of X_train----->', str(X_train.shape))
    print('Shape of X_val----->', str(X_val.shape))
    print('Shape of X_test----->', str(X_test.shape))
    return (X_train, X_val, X_test, y_train, y_val, y_test)

In [ ]:
def get_models():
    ANN = Sequential()
    ANN.add(Dense(128, input_dim = X_train.shape[1], activation = 'relu'))
    ANN.add(BatchNormalization())
    ANN.add(Dropout(0.2))
    ANN.add(Dense(64, activation='relu'))
    ANN.add(Dense(32, activation='relu'))
    ANN.add(Dense(16, activation='relu'))
    ANN.add(Dense(8, activation='relu'))
    ANN.add(Dense(len(train_it.class_indices), activation='softmax'))
    ANN.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    LR = LogisticRegression()

    KNN = KNeighborsClassifier(n_neighbors=50)

    SVM = SVC(kernel = 'linear')

    RF = RandomForestClassifier(n_estimators = 50)

    XGB = XGBClassifier(n_estimators = 50, use_label_encoder=False)

    return (ANN, LR, KNN, SVM, RF, XGB)

In [ ]:
def reshape_data(X_train, X_val, X_test):
    X_train = X_train.reshape(X_train.shape[0], -1)
    X_val = X_val.reshape(X_val.shape[0], -1)
    X_test = X_test.reshape(X_test.shape[0], -1)

    print("Shape after reshaping------->")
    print("X train------->", str(X_train.shape))
    print("X val-------->", str(X_val.shape))
    print("X test-------->", str(X_test.shape))

    return (X_train, X_val, X_test)

In [ ]:
def fit_ANN(model, X_train, y_train, X_val, y_test):
    es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)
    history = model.fit(X_train, y_train, validation_data=(X_val, y_test), epochs=10, verbose=1, callbacks=[es])
    return model

def fit_model(model, X_train, y_train):
    model.fit(X_train, y_train)
    return model

In [ ]:
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.models import Model
base_model = DenseNet121(include_top=False, weights='imagenet', input_shape=(SIZE_X, SIZE_Y, 3))

for layer in base_model.layers:
    layer.trainable = False

model = Model(inputs=base_model.input, outputs=base_model.layers[-1].output)
model.summary()

Model: "model_9"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_10 (InputLayer)          [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 zero_padding2d_2 (ZeroPadding2  (None, 230, 230, 3)  0          ['input_10[0][0]']               
 D)                                                                                               
                                                                                                  
 conv1/conv (Conv2D)            (None, 112, 112, 64  9408        ['zero_padding2d_2[0][0]']       
                                )                                                           

 conv2_block4_0_relu (Activatio  (None, 56, 56, 160)  0          ['conv2_block4_0_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv2_block4_1_conv (Conv2D)   (None, 56, 56, 128)  20480       ['conv2_block4_0_relu[0][0]']    
                                                                                                  
 conv2_block4_1_bn (BatchNormal  (None, 56, 56, 128)  512        ['conv2_block4_1_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv2_block4_1_relu (Activatio  (None, 56, 56, 128)  0          ['conv2_block4_1_bn[0][0]']      
 n)                                                                                               
          

                                                                                                  
 conv3_block2_0_relu (Activatio  (None, 28, 28, 160)  0          ['conv3_block2_0_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv3_block2_1_conv (Conv2D)   (None, 28, 28, 128)  20480       ['conv3_block2_0_relu[0][0]']    
                                                                                                  
 conv3_block2_1_bn (BatchNormal  (None, 28, 28, 128)  512        ['conv3_block2_1_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv3_block2_1_relu (Activatio  (None, 28, 28, 128)  0          ['conv3_block2_1_bn[0][0]']      
 n)       

 ization)                                                                                         
                                                                                                  
 conv3_block6_1_relu (Activatio  (None, 28, 28, 128)  0          ['conv3_block6_1_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv3_block6_2_conv (Conv2D)   (None, 28, 28, 32)   36864       ['conv3_block6_1_relu[0][0]']    
                                                                                                  
 conv3_block6_concat (Concatena  (None, 28, 28, 320)  0          ['conv3_block5_concat[0][0]',    
 te)                                                              'conv3_block6_2_conv[0][0]']    
                                                                                                  
 conv3_blo

 conv3_block10_concat (Concaten  (None, 28, 28, 448)  0          ['conv3_block9_concat[0][0]',    
 ate)                                                             'conv3_block10_2_conv[0][0]']   
                                                                                                  
 conv3_block11_0_bn (BatchNorma  (None, 28, 28, 448)  1792       ['conv3_block10_concat[0][0]']   
 lization)                                                                                        
                                                                                                  
 conv3_block11_0_relu (Activati  (None, 28, 28, 448)  0          ['conv3_block11_0_bn[0][0]']     
 on)                                                                                              
                                                                                                  
 conv3_block11_1_conv (Conv2D)  (None, 28, 28, 128)  57344       ['conv3_block11_0_relu[0][0]']   
          

...

                                                                                                  
 conv4_block20_1_conv (Conv2D)  (None, 14, 14, 128)  110592      ['conv4_block20_0_relu[0][0]']   
                                                                                                  
 conv4_block20_1_bn (BatchNorma  (None, 14, 14, 128)  512        ['conv4_block20_1_conv[0][0]']   
 lization)                                                                                        
                                                                                                  
 conv4_block20_1_relu (Activati  (None, 14, 14, 128)  0          ['conv4_block20_1_bn[0][0]']     
 on)                                                                                              
                                                                                                  
 conv4_block20_2_conv (Conv2D)  (None, 14, 14, 32)   36864       ['conv4_block20_1_relu[0][0]']   
          

 on)                                                                                              
                                                                                                  
 conv4_block24_2_conv (Conv2D)  (None, 14, 14, 32)   36864       ['conv4_block24_1_relu[0][0]']   
                                                                                                  
 conv4_block24_concat (Concaten  (None, 14, 14, 1024  0          ['conv4_block23_concat[0][0]',   
 ate)                           )                                 'conv4_block24_2_conv[0][0]']   
                                                                                                  
 pool4_bn (BatchNormalization)  (None, 14, 14, 1024  4096        ['conv4_block24_concat[0][0]']   
                                )                                                                 
                                                                                                  
 pool4_rel

 ization)                                                                                         
                                                                                                  
 conv5_block4_1_relu (Activatio  (None, 7, 7, 128)   0           ['conv5_block4_1_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv5_block4_2_conv (Conv2D)   (None, 7, 7, 32)     36864       ['conv5_block4_1_relu[0][0]']    
                                                                                                  
 conv5_block4_concat (Concatena  (None, 7, 7, 640)   0           ['conv5_block3_concat[0][0]',    
 te)                                                              'conv5_block4_2_conv[0][0]']    
                                                                                                  
 conv5_blo

 conv5_block8_concat (Concatena  (None, 7, 7, 768)   0           ['conv5_block7_concat[0][0]',    
 te)                                                              'conv5_block8_2_conv[0][0]']    
                                                                                                  
 conv5_block9_0_bn (BatchNormal  (None, 7, 7, 768)   3072        ['conv5_block8_concat[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv5_block9_0_relu (Activatio  (None, 7, 7, 768)   0           ['conv5_block9_0_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv5_block9_1_conv (Conv2D)   (None, 7, 7, 128)    98304       ['conv5_block9_0_relu[0][0]']    
          

 on)                                                                                              
                                                                                                  
 conv5_block13_1_conv (Conv2D)  (None, 7, 7, 128)    114688      ['conv5_block13_0_relu[0][0]']   
                                                                                                  
 conv5_block13_1_bn (BatchNorma  (None, 7, 7, 128)   512         ['conv5_block13_1_conv[0][0]']   
 lization)                                                                                        
                                                                                                  
 conv5_block13_1_relu (Activati  (None, 7, 7, 128)   0           ['conv5_block13_1_bn[0][0]']     
 on)                                                                                              
                                                                                                  
 conv5_blo

In [ ]:
X_train, X_val, X_test, y_train, y_val, y_test = get_features(model, train_it, validate_it)

Shape of X_train-----> (7000, 7, 7, 1024)
Shape of X_val-----> (1500, 7, 7, 1024)
Shape of X_test-----> (1500, 7, 7, 1024)


In [ ]:
X_train, X_val, X_test = reshape_data(X_train, X_val, X_test)
ANN, LR, KNN, SVM, RF, XGB = get_models()

from sklearn.model_selection import KFold

# Define number of splits for K-Fold cross-validation
k_folds = 5
kf = KFold(n_splits=k_folds, shuffle=True)

# Initialize lists to store performance metrics for each fold
accuracy_scores_ANN = []
f1_scores_ANN = []
accuracy_scores_LR = []
f1_scores_LR = []
accuracy_scores_KNN = []
f1_scores_KNN = []
accuracy_scores_SVM = []
f1_scores_SVM = []
accuracy_scores_RF = []
f1_scores_RF = []
accuracy_scores_XGB = []
f1_scores_XGB = []
# Perform K-Fold cross-validation
for train_index, val_index in kf.split(X_train):
    X_train_fold, X_val_fold = X_train[train_index], X_train[val_index]
    y_train_fold, y_val_fold = y_train[train_index], y_train[val_index]

    # Train ANN model
    ANN_fold = fit_ANN(ANN, X_train_fold, y_train_fold, X_val_fold, y_val_fold)
    # Evaluate ANN model
    accuracy_ANN = accuracy_score(y_test, np.argmax(ANN_fold.predict(X_test), axis=1))
    f1_ANN = f1_score(y_test, np.argmax(ANN_fold.predict(X_test), axis=1))
    accuracy_scores_ANN.append(accuracy_ANN)
    f1_scores_ANN.append(f1_ANN)

    # Train LR model
    LR_fold = fit_model(LR, X_train_fold, y_train_fold)
    # Evaluate LR model
    accuracy_LR = accuracy_score(y_test, LR_fold.predict(X_test))
    accuracy_scores_LR.append(accuracy_LR)
    f1_LR=f1_score(y_test, LR_fold.predict(X_test))
    f1_scores_LR.append(f1_LR)

    # Train KNN model
    KNN_fold = fit_model(KNN, X_train_fold, y_train_fold)
    # Evaluate KNN model
    accuracy_KNN = accuracy_score(y_test, KNN_fold.predict(X_test))
    accuracy_scores_KNN.append(accuracy_KNN)
    f1_KNN=f1_score(y_test, KNN_fold.predict(X_test))
    f1_scores_KNN.append(f1_KNN)

    # Train SVM model
    SVM_fold = fit_model(SVM, X_train_fold, y_train_fold)
    # Evaluate SVM model
    accuracy_SVM = accuracy_score(y_test, SVM_fold.predict(X_test))
    accuracy_scores_SVM.append(accuracy_SVM)
    f1_SVM=f1_score(y_test, SVM_fold.predict(X_test))
    f1_scores_SVM.append(f1_SVM)

    # Train RF model
    RF_fold = fit_model(RF, X_train_fold, y_train_fold)
    # Evaluate RF model
    accuracy_RF = accuracy_score(y_test, RF_fold.predict(X_test))
    accuracy_scores_RF.append(accuracy_RF)
    f1_RF=f1_score(y_test, RF_fold.predict(X_test))
    f1_scores_RF.append(f1_RF)

    # Train XGB model
    XGB_fold = fit_model(XGB, X_train_fold, y_train_fold)
    # Evaluate XGB model
    accuracy_XGB = accuracy_score(y_test, XGB_fold.predict(X_test))
    accuracy_scores_XGB.append(accuracy_XGB)
    f1_XGB=f1_score(y_test, XGB_fold.predict(X_test))
    f1_scores_XGB.append(f1_XGB)

Shape after reshaping------->
X train-------> (7000, 50176)
X val--------> (1500, 50176)
X test--------> (1500, 50176)
Epoch 1/10
175/175 [==============================] - 25s 139ms/step - loss: 0.2244 - accuracy: 0.9091 - val_loss: 0.2017 - val_accuracy: 0.9100
Epoch 2/10
175/175 [==============================] - 24s 140ms/step - loss: 0.0994 - accuracy: 0.9634 - val_loss: 0.1867 - val_accuracy: 0.9286
Epoch 3/10
175/175 [==============================] - 24s 138ms/step - loss: 0.0716 - accuracy: 0.9700 - val_loss: 0.2107 - val_accuracy: 0.9350
Epoch 4/10
175/175 [==============================] - 24s 136ms/step - loss: 0.0551 - accuracy: 0.9796 - val_loss: 0.0743 - val_accuracy: 0.9750
Epoch 5/10
175/175 [==============================] - 23s 129ms/step - loss: 0.0418 - accuracy: 0.9846 - val_loss: 0.1855 - val_accuracy: 0.9321
Epoch 6/10
175/175 [==============================] - 24s 138ms/step - loss: 0.0348 - accuracy: 0.9873 - val_loss: 0.1426 - val_accuracy: 0.9557
Epoch 7/10


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Epoch 1/10
175/175 [==============================] - 17s 99ms/step - loss: 0.0282 - accuracy: 0.9912 - val_loss: 0.0269 - val_accuracy: 0.9936
Epoch 2/10
175/175 [==============================] - 18s 100ms/step - loss: 0.0410 - accuracy: 0.9866 - val_loss: 0.0213 - val_accuracy: 0.9921
Epoch 3/10
175/175 [==============================] - 18s 102ms/step - loss: 0.0270 - accuracy: 0.9905 - val_loss: 0.0596 - val_accuracy: 0.9829
Epoch 4/10
175/175 [==============================] - 18s 103ms/step - loss: 0.0295 - accuracy: 0.9898 - val_loss: 0.1553 - val_accuracy: 0.9357
Epoch 5/10
175/175 [==============================] - 17s 99ms/step - loss: 0.0221 - accuracy: 0.9939 - val_loss: 0.0356 - val_accuracy: 0.9879
Epoch 6/10
175/175 [==============================] - 17s 99ms/step - loss: 0.0151 - accuracy: 0.9945 - val_loss: 0.0454 - val_accuracy: 0.9886
Epoch 7/10
175/175 [==============================] - 18s 105ms/step - loss: 0.0206 - accuracy: 0.9923 - val_loss: 0.7685 - val_accur

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Epoch 1/10
175/175 [==============================] - 18s 105ms/step - loss: 0.0337 - accuracy: 0.9879 - val_loss: 0.1439 - val_accuracy: 0.9736
Epoch 2/10
175/175 [==============================] - 18s 103ms/step - loss: 0.0262 - accuracy: 0.9914 - val_loss: 1.7856 - val_accuracy: 0.6607
Epoch 3/10
175/175 [==============================] - 18s 103ms/step - loss: 0.0218 - accuracy: 0.9916 - val_loss: 0.4475 - val_accuracy: 0.9643
Epoch 4/10
175/175 [==============================] - 18s 103ms/step - loss: 0.0240 - accuracy: 0.9912 - val_loss: 0.5791 - val_accuracy: 0.9421
Epoch 5/10
175/175 [==============================] - 18s 102ms/step - loss: 0.0239 - accuracy: 0.9907 - val_loss: 0.2215 - val_accuracy: 0.9921
Epoch 6/10
175/175 [==============================] - 17s 99ms/step - loss: 0.0116 - accuracy: 0.9957 - val_loss: 0.3147 - val_accuracy: 0.9971
Epoch 6: early stopping


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Epoch 1/10
175/175 [==============================] - 19s 108ms/step - loss: 0.0165 - accuracy: 0.9946 - val_loss: 0.0144 - val_accuracy: 0.9936
Epoch 2/10
175/175 [==============================] - 19s 106ms/step - loss: 0.0118 - accuracy: 0.9966 - val_loss: 9.9907e-04 - val_accuracy: 1.0000
Epoch 3/10
175/175 [==============================] - 19s 107ms/step - loss: 0.0155 - accuracy: 0.9948 - val_loss: 0.0702 - val_accuracy: 0.9714
Epoch 4/10
175/175 [==============================] - 18s 102ms/step - loss: 0.0080 - accuracy: 0.9973 - val_loss: 0.0022 - val_accuracy: 0.9993
Epoch 5/10
175/175 [==============================] - 17s 100ms/step - loss: 0.0177 - accuracy: 0.9952 - val_loss: 0.0124 - val_accuracy: 0.9950
Epoch 6/10
175/175 [==============================] - 17s 99ms/step - loss: 0.0108 - accuracy: 0.9957 - val_loss: 0.0152 - val_accuracy: 0.9943
Epoch 7/10
175/175 [==============================] - 18s 100ms/step - loss: 0.0135 - accuracy: 0.9964 - val_loss: 0.0077 - val

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Epoch 1/10
175/175 [==============================] - 19s 106ms/step - loss: 0.0106 - accuracy: 0.9964 - val_loss: 0.0014 - val_accuracy: 0.9993
Epoch 2/10
175/175 [==============================] - 19s 109ms/step - loss: 0.0146 - accuracy: 0.9950 - val_loss: 0.0157 - val_accuracy: 0.9950
Epoch 3/10
175/175 [==============================] - 18s 103ms/step - loss: 0.0133 - accuracy: 0.9946 - val_loss: 0.0039 - val_accuracy: 0.9986
Epoch 4/10
175/175 [==============================] - 18s 105ms/step - loss: 0.0120 - accuracy: 0.9962 - val_loss: 0.0073 - val_accuracy: 0.9964
Epoch 5/10
175/175 [==============================] - 18s 103ms/step - loss: 0.0090 - accuracy: 0.9971 - val_loss: 0.0021 - val_accuracy: 1.0000
Epoch 6/10
175/175 [==============================] - 17s 98ms/step - loss: 0.0051 - accuracy: 0.9987 - val_loss: 0.0073 - val_accuracy: 0.9979
Epoch 6: early stopping


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
average_accuracy_ANN = np.mean(accuracy_scores_ANN)
average_f1_ANN = np.mean(f1_scores_ANN)
average_accuracy_LR = np.mean(accuracy_scores_LR)
average_f1_LR = np.mean(f1_scores_LR)
average_accuracy_KNN = np.mean(accuracy_scores_KNN)
average_f1_KNN = np.mean(f1_scores_KNN)
average_accuracy_SVM = np.mean(accuracy_scores_SVM)
average_f1_SVM = np.mean(f1_scores_SVM)
average_accuracy_RF = np.mean(accuracy_scores_RF)
average_f1_RF = np.mean(f1_scores_RF)
average_accuracy_XGB = np.mean(accuracy_scores_XGB)
average_f1_XGB = np.mean(f1_scores_XGB)

In [ ]:
print(accuracy_scores_ANN)

[0.968, 0.7706666666666667, 0.984, 0.982, 0.9806666666666667]


In [ ]:
print(accuracy_scores_LR)

[0.9813333333333333, 0.9673333333333334, 0.9773333333333334, 0.9793333333333333, 0.9713333333333334]


In [ ]:
print(accuracy_scores_KNN)

[0.8846666666666667, 0.8853333333333333, 0.888, 0.882, 0.8873333333333333]


In [ ]:
print(accuracy_scores_SVM)

[0.982, 0.9766666666666667, 0.9793333333333333, 0.9813333333333333, 0.9726666666666667]


In [ ]:
print(accuracy_scores_RF)

[0.9466666666666667, 0.9473333333333334, 0.9406666666666667, 0.9393333333333334, 0.9433333333333334]


In [ ]:
print(accuracy_scores_XGB)

[0.9693333333333334, 0.9633333333333334, 0.964, 0.9653333333333334, 0.9626666666666667]


In [ ]:
print(f1_scores_ANN)

[0.9691912708600771, 0.7054794520547945, 0.9840425531914894, 0.9823644676681906, 0.9811074918566774]


In [ ]:
print(f1_scores_LR)

[0.9815303430079156, 0.9676140118968936, 0.9775725593667546, 0.9794293297942933, 0.9715796430931923]


In [ ]:
print(f1_scores_KNN)

[0.8950879320800484, 0.8955042527339003, 0.8974358974358975, 0.8925318761384335, 0.8973891924711597]


In [ ]:
print(f1_scores_SVM)

[0.982178217821782, 0.9768976897689768, 0.9795649307844428, 0.9815059445178336, 0.9729015201586253]


In [ ]:
print(f1_scores_RF)

[0.9480519480519479, 0.9485342019543972, 0.9423948220064724, 0.9412524209167203, 0.9451258876694643]


In [ ]:
print(f1_scores_XGB)

[0.9701298701298702, 0.9642160052049447, 0.9647979139504562, 0.9661016949152542, 0.9636363636363636]


In [ ]:
print("Average Accuracy for ANN model:", average_accuracy_ANN)

Average Accuracy for ANN model: 0.9370666666666667


In [ ]:
print("Average F1 for ANN model:", average_f1_ANN)

Average F1 for ANN model: 0.9244370471262459


In [ ]:
print("Average Accuracy for LR model:", average_accuracy_LR)

Average Accuracy for LR model: 0.9753333333333334


In [ ]:
print("Average F1 for LR model:", average_f1_LR)

Average F1 for LR model: 0.9755451774318098


In [ ]:
print("Average Accuracy for KNN model:", average_accuracy_KNN)

Average Accuracy for KNN model: 0.8854666666666666


In [ ]:
print("Average F1 for KNN model:", average_f1_KNN)

Average F1 for KNN model: 0.8955898301718879


In [ ]:
print("Average Accuracy for SVM model:", average_accuracy_SVM)

Average Accuracy for SVM model: 0.9784


In [ ]:
print("Average F1 for SVM model:", average_f1_SVM)

Average F1 for SVM model: 0.978609660610332


In [ ]:
print("Average Accuracy for RF model:", average_accuracy_RF)

Average Accuracy for RF model: 0.9434666666666667


In [ ]:
print("Average F1 for RF model:", average_f1_RF)

Average F1 for RF model: 0.9450718561198004


In [ ]:
print("Average Accuracy for XGB model:", average_accuracy_XGB)

Average Accuracy for XGB model: 0.9649333333333333


In [ ]:
print("Average F1 for XGB model:", average_f1_XGB)

Average F1 for XGB model: 0.9657763695673778


###**EfficientNet**

Import all libraries required

In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score, confusion_matrix
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout
from keras.models import Sequential
import tensorflow as tf
import matplotlib.pyplot as plt
import cv2
from tensorflow.keras.layers import BatchNormalization
from keras.models import Model, Sequential
from keras.applications.xception import Xception
from keras.applications import *
import matplotlib.pyplot as plt
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline
from PIL import Image
import random
import os
import cv2
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from keras.callbacks import EarlyStopping
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm
from sklearn.decomposition import PCA

In [ ]:
data_dir = "colon_image_sets"
SIZE_X = SIZE_Y = 224

datagen = tf.keras.preprocessing.image.ImageDataGenerator(validation_split = 0.3)

train_it = datagen.flow_from_directory(data_dir,
                                       class_mode = "categorical",
                                       target_size = (SIZE_X,SIZE_Y),
                                       color_mode="rgb",
                                       batch_size = 12,
                                       shuffle = False,
                                       subset='training',
                                       seed = 42)

validate_it = datagen.flow_from_directory(data_dir,
                                       class_mode = "categorical",
                                       target_size = (SIZE_X, SIZE_Y),
                                       color_mode="rgb",
                                       batch_size = 12,
                                       shuffle = False,
                                       subset='validation',
                                       seed = 42)

Found 7000 images belonging to 2 classes.
Found 3000 images belonging to 2 classes.


In [ ]:
def get_features(base_model, train, validate):
    X_train = base_model.predict(train)
    y_train = train.classes

    X_val = base_model.predict(validate)
    y_val = validate.classes

    X_val, X_test, y_val, y_test = train_test_split(X_val, y_val, test_size = 0.5, shuffle = True)
    print('Shape of X_train----->', str(X_train.shape))
    print('Shape of X_val----->', str(X_val.shape))
    print('Shape of X_test----->', str(X_test.shape))
    return (X_train, X_val, X_test, y_train, y_val, y_test)

In [ ]:
def get_models():
    ANN = Sequential()
    ANN.add(Dense(128, input_dim = X_train.shape[1], activation = 'relu'))
    ANN.add(BatchNormalization())
    ANN.add(Dropout(0.2))
    ANN.add(Dense(64, activation='relu'))
    ANN.add(Dense(32, activation='relu'))
    ANN.add(Dense(16, activation='relu'))
    ANN.add(Dense(8, activation='relu'))
    ANN.add(Dense(len(train_it.class_indices), activation='softmax'))
    ANN.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    LR = LogisticRegression()

    KNN = KNeighborsClassifier(n_neighbors=50)

    SVM = SVC(kernel = 'linear')

    RF = RandomForestClassifier(n_estimators = 50)

    XGB = XGBClassifier(n_estimators = 50, use_label_encoder=False)

    return (ANN, LR, KNN, SVM, RF, XGB)

In [ ]:
def reshape_data(X_train, X_val, X_test):
    X_train = X_train.reshape(X_train.shape[0], -1)
    X_val = X_val.reshape(X_val.shape[0], -1)
    X_test = X_test.reshape(X_test.shape[0], -1)

    print("Shape after reshaping------->")
    print("X train------->", str(X_train.shape))
    print("X val-------->", str(X_val.shape))
    print("X test-------->", str(X_test.shape))

    return (X_train, X_val, X_test)

In [ ]:
def fit_ANN(model, X_train, y_train, X_val, y_test):
    es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)
    history = model.fit(X_train, y_train, validation_data=(X_val, y_test), epochs=10, verbose=1, callbacks=[es])
    return model

def fit_model(model, X_train, y_train):
    model.fit(X_train, y_train)
    return model

In [ ]:
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.models import Model

base_model = EfficientNetB0(include_top=False, weights='imagenet', input_shape=(SIZE_X, SIZE_Y, 3))

for layer in base_model.layers:
    layer.trainable = False

model = Model(inputs=base_model.input, outputs=base_model.layers[-1].output)
model.summary()

Model: "model_10"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_11 (InputLayer)          [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 rescaling_1 (Rescaling)        (None, 224, 224, 3)  0           ['input_11[0][0]']               
                                                                                                  
 normalization_1 (Normalization  (None, 224, 224, 3)  7          ['rescaling_1[0][0]']            
 )                                                                                                
                                                                                           

                                                                                                  
 block2a_project_bn (BatchNorma  (None, 56, 56, 24)  96          ['block2a_project_conv[0][0]']   
 lization)                                                                                        
                                                                                                  
 block2b_expand_conv (Conv2D)   (None, 56, 56, 144)  3456        ['block2a_project_bn[0][0]']     
                                                                                                  
 block2b_expand_bn (BatchNormal  (None, 56, 56, 144)  576        ['block2b_expand_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 block2b_expand_activation (Act  (None, 56, 56, 144)  0          ['block2b_expand_bn[0][0]']      
 ivation) 

 ization)                                                                                         
                                                                                                  
 block3b_expand_activation (Act  (None, 28, 28, 240)  0          ['block3b_expand_bn[0][0]']      
 ivation)                                                                                         
                                                                                                  
 block3b_dwconv (DepthwiseConv2  (None, 28, 28, 240)  6000       ['block3b_expand_activation[0][0]
 D)                                                              ']                               
                                                                                                  
 block3b_bn (BatchNormalization  (None, 28, 28, 240)  960        ['block3b_dwconv[0][0]']         
 )                                                                                                
          

                                                                                                  
 block4b_bn (BatchNormalization  (None, 14, 14, 480)  1920       ['block4b_dwconv[0][0]']         
 )                                                                                                
                                                                                                  
 block4b_activation (Activation  (None, 14, 14, 480)  0          ['block4b_bn[0][0]']             
 )                                                                                                
                                                                                                  
 block4b_se_squeeze (GlobalAver  (None, 480)         0           ['block4b_activation[0][0]']     
 agePooling2D)                                                                                    
                                                                                                  
 block4b_s

 )                                                                                                
                                                                                                  
 block5a_se_squeeze (GlobalAver  (None, 480)         0           ['block5a_activation[0][0]']     
 agePooling2D)                                                                                    
                                                                                                  
 block5a_se_reshape (Reshape)   (None, 1, 1, 480)    0           ['block5a_se_squeeze[0][0]']     
                                                                                                  
 block5a_se_reduce (Conv2D)     (None, 1, 1, 20)     9620        ['block5a_se_reshape[0][0]']     
                                                                                                  
 block5a_se_expand (Conv2D)     (None, 1, 1, 480)    10080       ['block5a_se_reduce[0][0]']      
          

                                                                                                  
 block5c_se_excite (Multiply)   (None, 14, 14, 672)  0           ['block5c_activation[0][0]',     
                                                                  'block5c_se_expand[0][0]']      
                                                                                                  
 block5c_project_conv (Conv2D)  (None, 14, 14, 112)  75264       ['block5c_se_excite[0][0]']      
                                                                                                  
 block5c_project_bn (BatchNorma  (None, 14, 14, 112)  448        ['block5c_project_conv[0][0]']   
 lization)                                                                                        
                                                                                                  
 block5c_drop (Dropout)         (None, 14, 14, 112)  0           ['block5c_project_bn[0][0]']     
          

 lization)                                                                                        
                                                                                                  
 block6b_drop (Dropout)         (None, 7, 7, 192)    0           ['block6b_project_bn[0][0]']     
                                                                                                  
 block6b_add (Add)              (None, 7, 7, 192)    0           ['block6b_drop[0][0]',           
                                                                  'block6a_project_bn[0][0]']     
                                                                                                  
 block6c_expand_conv (Conv2D)   (None, 7, 7, 1152)   221184      ['block6b_add[0][0]']            
                                                                                                  
 block6c_expand_bn (BatchNormal  (None, 7, 7, 1152)  4608        ['block6c_expand_conv[0][0]']    
 ization) 

                                                                  'block6c_add[0][0]']            
                                                                                                  
 block7a_expand_conv (Conv2D)   (None, 7, 7, 1152)   221184      ['block6d_add[0][0]']            
                                                                                                  
 block7a_expand_bn (BatchNormal  (None, 7, 7, 1152)  4608        ['block7a_expand_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 block7a_expand_activation (Act  (None, 7, 7, 1152)  0           ['block7a_expand_bn[0][0]']      
 ivation)                                                                                         
                                                                                                  
 block7a_d

In [ ]:
X_train, X_val, X_test, y_train, y_val, y_test = get_features(model, train_it, validate_it)

Shape of X_train-----> (7000, 7, 7, 1280)
Shape of X_val-----> (1500, 7, 7, 1280)
Shape of X_test-----> (1500, 7, 7, 1280)


In [ ]:
X_train, X_val, X_test = reshape_data(X_train, X_val, X_test)
ANN, LR, KNN, SVM, RF, XGB = get_models()

from sklearn.model_selection import KFold

# Define number of splits for K-Fold cross-validation
k_folds = 5
kf = KFold(n_splits=k_folds, shuffle=True)

# Initialize lists to store performance metrics for each fold
accuracy_scores_ANN = []
f1_scores_ANN = []
accuracy_scores_LR = []
f1_scores_LR = []
accuracy_scores_KNN = []
f1_scores_KNN = []
accuracy_scores_SVM = []
f1_scores_SVM = []
accuracy_scores_RF = []
f1_scores_RF = []
accuracy_scores_XGB = []
f1_scores_XGB = []
# Perform K-Fold cross-validation
for train_index, val_index in kf.split(X_train):
    X_train_fold, X_val_fold = X_train[train_index], X_train[val_index]
    y_train_fold, y_val_fold = y_train[train_index], y_train[val_index]

    # Train ANN model
    ANN_fold = fit_ANN(ANN, X_train_fold, y_train_fold, X_val_fold, y_val_fold)
    # Evaluate ANN model
    accuracy_ANN = accuracy_score(y_test, np.argmax(ANN_fold.predict(X_test), axis=1))
    f1_ANN = f1_score(y_test, np.argmax(ANN_fold.predict(X_test), axis=1))
    accuracy_scores_ANN.append(accuracy_ANN)
    f1_scores_ANN.append(f1_ANN)

    # Train LR model
    LR_fold = fit_model(LR, X_train_fold, y_train_fold)
    # Evaluate LR model
    accuracy_LR = accuracy_score(y_test, LR_fold.predict(X_test))
    accuracy_scores_LR.append(accuracy_LR)
    f1_LR=f1_score(y_test, LR_fold.predict(X_test))
    f1_scores_LR.append(f1_LR)

    # Train KNN model
    KNN_fold = fit_model(KNN, X_train_fold, y_train_fold)
    # Evaluate KNN model
    accuracy_KNN = accuracy_score(y_test, KNN_fold.predict(X_test))
    accuracy_scores_KNN.append(accuracy_KNN)
    f1_KNN=f1_score(y_test, KNN_fold.predict(X_test))
    f1_scores_KNN.append(f1_KNN)

    # Train SVM model
    SVM_fold = fit_model(SVM, X_train_fold, y_train_fold)
    # Evaluate SVM model
    accuracy_SVM = accuracy_score(y_test, SVM_fold.predict(X_test))
    accuracy_scores_SVM.append(accuracy_SVM)
    f1_SVM=f1_score(y_test, SVM_fold.predict(X_test))
    f1_scores_SVM.append(f1_SVM)

    # Train RF model
    RF_fold = fit_model(RF, X_train_fold, y_train_fold)
    # Evaluate RF model
    accuracy_RF = accuracy_score(y_test, RF_fold.predict(X_test))
    accuracy_scores_RF.append(accuracy_RF)
    f1_RF=f1_score(y_test, RF_fold.predict(X_test))
    f1_scores_RF.append(f1_RF)

    # Train XGB model
    XGB_fold = fit_model(XGB, X_train_fold, y_train_fold)
    # Evaluate XGB model
    accuracy_XGB = accuracy_score(y_test, XGB_fold.predict(X_test))
    accuracy_scores_XGB.append(accuracy_XGB)
    f1_XGB=f1_score(y_test, XGB_fold.predict(X_test))
    f1_scores_XGB.append(f1_XGB)

Shape after reshaping------->
X train-------> (7000, 62720)
X val--------> (1500, 62720)
X test--------> (1500, 62720)
Epoch 1/10
175/175 [==============================] - 12s 63ms/step - loss: 0.0784 - accuracy: 0.9723 - val_loss: 0.0080 - val_accuracy: 0.9979
Epoch 2/10
175/175 [==============================] - 11s 63ms/step - loss: 0.0119 - accuracy: 0.9964 - val_loss: 0.0314 - val_accuracy: 0.9857
Epoch 3/10
175/175 [==============================] - 11s 63ms/step - loss: 0.0140 - accuracy: 0.9948 - val_loss: 0.0100 - val_accuracy: 0.9964
Epoch 4/10
175/175 [==============================] - 11s 61ms/step - loss: 0.0046 - accuracy: 0.9984 - val_loss: 0.0072 - val_accuracy: 0.9979
Epoch 5/10
175/175 [==============================] - 11s 61ms/step - loss: 0.0090 - accuracy: 0.9973 - val_loss: 0.0054 - val_accuracy: 0.9979
Epoch 6/10
175/175 [==============================] - 11s 61ms/step - loss: 0.0071 - accuracy: 0.9979 - val_loss: 0.0055 - val_accuracy: 0.9986
Epoch 7/10
175/17

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Epoch 1/10
175/175 [==============================] - 12s 66ms/step - loss: 0.0042 - accuracy: 0.9989 - val_loss: 5.1377e-04 - val_accuracy: 1.0000
Epoch 2/10
175/175 [==============================] - 13s 71ms/step - loss: 0.0022 - accuracy: 0.9991 - val_loss: 1.7710e-04 - val_accuracy: 1.0000
Epoch 3/10
175/175 [==============================] - 12s 67ms/step - loss: 0.0017 - accuracy: 0.9996 - val_loss: 8.6887e-05 - val_accuracy: 1.0000
Epoch 4/10
175/175 [==============================] - 12s 66ms/step - loss: 0.0072 - accuracy: 0.9980 - val_loss: 0.0037 - val_accuracy: 0.9986
Epoch 5/10
175/175 [==============================] - 12s 66ms/step - loss: 0.0047 - accuracy: 0.9982 - val_loss: 0.0124 - val_accuracy: 0.9964
Epoch 6/10
175/175 [==============================] - 12s 70ms/step - loss: 0.0025 - accuracy: 0.9995 - val_loss: 0.0018 - val_accuracy: 0.9993
Epoch 7/10
175/175 [==============================] - 12s 69ms/step - loss: 0.0037 - accuracy: 0.9989 - val_loss: 0.0166 - v

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Epoch 1/10
175/175 [==============================] - 11s 62ms/step - loss: 0.0012 - accuracy: 0.9995 - val_loss: 0.0055 - val_accuracy: 0.9986
Epoch 2/10
175/175 [==============================] - 11s 62ms/step - loss: 0.0058 - accuracy: 0.9984 - val_loss: 0.0091 - val_accuracy: 0.9979
Epoch 3/10
175/175 [==============================] - 11s 61ms/step - loss: 0.0032 - accuracy: 0.9989 - val_loss: 3.6051e-04 - val_accuracy: 1.0000
Epoch 4/10
175/175 [==============================] - 11s 62ms/step - loss: 5.2539e-04 - accuracy: 0.9998 - val_loss: 0.0023 - val_accuracy: 0.9993
Epoch 5/10
175/175 [==============================] - 11s 63ms/step - loss: 5.0388e-04 - accuracy: 0.9996 - val_loss: 0.0029 - val_accuracy: 0.9993
Epoch 6/10
175/175 [==============================] - 11s 62ms/step - loss: 8.7645e-04 - accuracy: 0.9996 - val_loss: 0.0043 - val_accuracy: 0.9993
Epoch 7/10
175/175 [==============================] - 11s 63ms/step - loss: 0.0021 - accuracy: 0.9991 - val_loss: 0.0042

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Epoch 1/10
175/175 [==============================] - 11s 61ms/step - loss: 0.0057 - accuracy: 0.9979 - val_loss: 0.0073 - val_accuracy: 0.9986
Epoch 2/10
175/175 [==============================] - 11s 61ms/step - loss: 0.0065 - accuracy: 0.9982 - val_loss: 0.0326 - val_accuracy: 0.9993
Epoch 3/10
175/175 [==============================] - 11s 63ms/step - loss: 9.7679e-04 - accuracy: 0.9996 - val_loss: 0.0486 - val_accuracy: 0.9993
Epoch 4/10
175/175 [==============================] - 11s 61ms/step - loss: 0.0012 - accuracy: 0.9996 - val_loss: 0.0452 - val_accuracy: 0.9993
Epoch 5/10
175/175 [==============================] - 11s 61ms/step - loss: 0.0020 - accuracy: 0.9995 - val_loss: 0.0259 - val_accuracy: 0.9986
Epoch 6/10
175/175 [==============================] - 11s 61ms/step - loss: 0.0028 - accuracy: 0.9991 - val_loss: 0.0324 - val_accuracy: 0.9993
Epoch 6: early stopping


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Epoch 1/10
175/175 [==============================] - 11s 64ms/step - loss: 0.0024 - accuracy: 0.9995 - val_loss: 0.0011 - val_accuracy: 1.0000
Epoch 2/10
175/175 [==============================] - 11s 61ms/step - loss: 9.7504e-04 - accuracy: 0.9996 - val_loss: 4.7914e-04 - val_accuracy: 1.0000
Epoch 3/10
175/175 [==============================] - 11s 61ms/step - loss: 0.0011 - accuracy: 0.9995 - val_loss: 2.9516e-05 - val_accuracy: 1.0000
Epoch 4/10
175/175 [==============================] - 11s 62ms/step - loss: 4.1690e-04 - accuracy: 0.9998 - val_loss: 7.7938e-05 - val_accuracy: 1.0000
Epoch 5/10
175/175 [==============================] - 11s 62ms/step - loss: 1.2157e-04 - accuracy: 1.0000 - val_loss: 0.0020 - val_accuracy: 0.9993
Epoch 6/10
175/175 [==============================] - 11s 61ms/step - loss: 2.3419e-04 - accuracy: 0.9998 - val_loss: 2.8921e-05 - val_accuracy: 1.0000
Epoch 7/10
175/175 [==============================] - 11s 62ms/step - loss: 0.0028 - accuracy: 0.9995 - 

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
average_accuracy_ANN = np.mean(accuracy_scores_ANN)
average_f1_ANN = np.mean(f1_scores_ANN)
average_accuracy_LR = np.mean(accuracy_scores_LR)
average_f1_LR = np.mean(f1_scores_LR)
average_accuracy_KNN = np.mean(accuracy_scores_KNN)
average_f1_KNN = np.mean(f1_scores_KNN)
average_accuracy_SVM = np.mean(accuracy_scores_SVM)
average_f1_SVM = np.mean(f1_scores_SVM)
average_accuracy_RF = np.mean(accuracy_scores_RF)
average_f1_RF = np.mean(f1_scores_RF)
average_accuracy_XGB = np.mean(accuracy_scores_XGB)
average_f1_XGB = np.mean(f1_scores_XGB)

In [ ]:
print(accuracy_scores_ANN)

[0.9946666666666667, 0.998, 0.998, 0.9993333333333333, 1.0]


In [ ]:
print(accuracy_scores_LR)

[0.9993333333333333, 0.9986666666666667, 1.0, 0.9993333333333333, 0.998]


In [ ]:
print(accuracy_scores_KNN)

[0.9626666666666667, 0.9593333333333334, 0.9613333333333334, 0.9613333333333334, 0.9653333333333334]


In [ ]:
print(accuracy_scores_SVM)

[0.9993333333333333, 1.0, 1.0, 0.9993333333333333, 0.9993333333333333]


In [ ]:
print(accuracy_scores_RF)

[0.968, 0.9713333333333334, 0.9706666666666667, 0.974, 0.974]


In [ ]:
print(accuracy_scores_XGB)

[0.9846666666666667, 0.9853333333333333, 0.9873333333333333, 0.9833333333333333, 0.9826666666666667]


In [ ]:
print(f1_scores_ANN)

[0.9946452476572958, 0.9979825151311366, 0.9979825151311366, 0.9993265993265993, 1.0]


In [ ]:
print(f1_scores_LR)

[0.9993275050437123, 0.9986522911051213, 1.0, 0.9993265993265993, 0.997979797979798]


In [ ]:
print(f1_scores_KNN)

[0.9614855570839064, 0.9582477754962354, 0.9600550964187327, 0.9600550964187327, 0.9641873278236914]


In [ ]:
print(f1_scores_SVM)

[0.9993275050437123, 1.0, 1.0, 0.9993265993265993, 0.9993275050437123]


In [ ]:
print(f1_scores_RF)

[0.967741935483871, 0.9709263015551046, 0.9702702702702702, 0.9736664415935178, 0.9737373737373738]


In [ ]:
print(f1_scores_XGB)

[0.9845117845117846, 0.9851951547779273, 0.9872053872053872, 0.9831195138419987, 0.98252688172043]


In [ ]:
print("Average Accuracy for ANN model:", average_accuracy_ANN)

Average Accuracy for ANN model: 0.998


In [ ]:
print("Average F1 for ANN model:", average_f1_ANN)

Average F1 for ANN model: 0.9979873754492337


In [ ]:
print("Average Accuracy for LR model:", average_accuracy_LR)

Average Accuracy for LR model: 0.9990666666666668


In [ ]:
print("Average F1 for LR model:", average_f1_LR)

Average F1 for LR model: 0.9990572386910461


In [ ]:
print("Average Accuracy for KNN model:", average_accuracy_KNN)

Average Accuracy for KNN model: 0.9620000000000001


In [ ]:
print("Average F1 for KNN model:", average_f1_KNN)

Average F1 for KNN model: 0.9608061706482596


In [ ]:
print("Average Accuracy for SVM model:", average_accuracy_SVM)


Average Accuracy for SVM model: 0.9996


In [ ]:
print("Average F1 for SVM model:", average_f1_SVM)

Average F1 for SVM model: 0.9995963218828049


In [ ]:
print("Average Accuracy for RF model:", average_accuracy_RF)

Average Accuracy for RF model: 0.9716000000000001


In [ ]:
print("Average F1 for RF model:", average_f1_RF)

Average F1 for RF model: 0.9712684645280275


In [ ]:
print("Average Accuracy for XGB model:", average_accuracy_XGB)

Average Accuracy for XGB model: 0.9846666666666666


In [ ]:
print("Average F1 for XGB model:", average_f1_XGB)

Average F1 for XGB model: 0.9845117444115055
